# Laboratory 3: Deep Learning and Modeling Materials

We start with the prerequisites for the lab. It's always good to put this in at the beginning.

In [ ]:
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
from scipy.sparse import spdiags,linalg,eye
from tqdm import tqdm
import imageio
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Sampler, BatchSampler, Dataset, DataLoader, Subset, SubsetRandomSampler, random_split

In [ ]:
%pip install torchmetrics
from torchmetrics import Accuracy

## Project Part 1 : Ising model

No numerical physics class would be complete without doing the Ising model. Howevr, because we want to be starte of the art on this. We are going to do the Ising model in way that is a little bit different than what you have done in the past. We are going to do the Ising model with Deep learning. The first part of this project will largely follow work done in this nature paper

https://arxiv.org/abs/1605.01735

This captures much of the core ideas that we want in building simulators, and, of course highlights the basic scheme that we get in an Ising model.

Our first step is to setup the Ising model

### Step 1.1: The lattice Ising Model


The Ising model is a mathematical model used to describe the behavior of a collection of interacting magnetic moments, such as atoms in a solid or spins in a lattice. The model is named after Ernst Ising, a physicist who first proposed it in 1925.

#### 1.1.1   Setting up Model

In the Ising model, each magnetic moment is represented by a spin variable, which can take on the values of +1 or -1. The model assumes that these spins interact only with their nearest neighbors in a lattice, and that the interaction between them is influenced by an external magnetic field.

The Hamiltonian of the Ising model describes the energy of the system, and is given by:

$$
H = -J \sum_{i,j} s_{i} s_{j} - \mu_{b} \sum_{j} B_{j} s_{j}
$$

where J is the exchange interaction strength between neighboring spins, $\mu$ is the magnetic moment of each spin, $B_{j}$ is the external magnetic field, and the first sum is taken over all pairs of nearest-neighbor spins in the lattice.

The Ising model exhibits a phase transition, which is a sudden change in the behavior of the system as a parameter is varied. In the absence of an external magnetic field (B=0), the model exhibits a phase transition at a critical temperature known as the Curie temperature (Tc). Below the critical temperature, the system exhibits long-range order, with all spins aligning in the same direction, resulting in a net magnetization. Above the critical temperature, the system becomes disordered, with spins pointing in random directions, and the net magnetization vanishes.

The critical behavior of the Ising model at the phase transition is described by universal scaling laws, which are independent of the microscopic details of the system. These scaling laws have been used to study a wide range of physical systems, including magnets, fluids, and even social networks.

As a first step write (split the below code into multiple steps to build the concept and guide the the student)

As an example of how we will setup the Ising model, see the initialize code.

In [ ]:
np.random.seed(20)

def initialize(N):
    state = 2*np.random.randint(2, size=(N,N))-1
    return state

N=4
test=initialize(N)
print(test)

Ok, so now that we have done that let's define a Hamiltonian that will output the energy of the above grid. For this grid, we will use wrap-around (video-game) coordinates whereby the coordinate for $j+1 \forall j \geq N j+1\rightarrow j \mod N$ or in other words the in a 4x4 model the spin at x=0 y=2 can contribute to the x=3 y=2 neighbor spin the x-axis.

In [ ]:
#QUESTION
def hamiltonian(iArr,N):
    return energy

hamiltonian(test,N)
#print energy should be 4

The above energy with the random seed should give us a value of 4.

Now we would like to come up with a strategy to evolve the spin configurations of the Ising model. To do this we are going to follow a Markov Chain Monte-Carlo Proposal strategy.  For this write a Metropolis algorithm that

 * Flips the spin of the i,j-th element of the grid
 * Computes $\Delta H=E_{\rm after}-E_{\rm before}$ the change in energy for that element
 * Updates the spin flip with probability $p < e^{-\frac{\Delta H}{k_{b} T} }$

Note that we often define temperature in the Ising model using a variable $\beta=\frac{1}{k_{b}T}$. The update need not be temperature.  Additionally, to make our units extra simple we will set the Boltzman Constant $k_{b}=1$.

In [ ]:
def flip(i,j,iArr,Beta,N):
    return

print("Before Flip:\n",test)
flip(2,1,test,1,N)
print("After Flip:\n",test)
#the 2,1 element should go from -1 to 1

Now define the magnetization of the system. This is just the sum over *all* the spins in the array:

$$
M=\sum_{i\in {\rm Lattice}} \sigma_{i}
$$

Write a function to do this:

In [ ]:
def mag(iArr):
    return 
print("magnetization:",mag(test))#Should be zero if you flipped 2,1 correctly, otherwise from raw is -2

Write a function that tries to flip every element in the array in random order.  Use the functions above to help you.

In [ ]:
def update_rand(iArr,N,TM1):
    return

Finally, we can put it all together by adding a plotting function that can allow us to make a viedeo of the phase transition.

In [ ]:
from IPython.display import Image

def mapPlot(ax,fig, iArr, i, N, images):
    plt.cla()
    X, Y = np.meshgrid(range(N), range(N))
    #plt.setp(sp.get_yticklabels(), visible=False)
    #plt.setp(sp.get_xticklabels(), visible=False)
    ax.pcolormesh(X, Y, iArr, cmap=plt.cm.RdBu);
    ax.text(0.6, 0.3,'Time=%d'%i,fontdict={'size': 24, 'color':  'red'})#; plt.axis('tight')
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image  = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(image)

def runTemp(iT,iN,images,fig,ax,eqSteps=500,mcSteps=500):
    pArr = initialize(iN)         # initialise
    beta=1.0/iT
    for i in range(eqSteps):         # equilibrate
        update_rand(pArr, iN, beta)

    for i in range(mcSteps):
        update_rand(pArr, iN, beta)
        Ene = hamiltonian(pArr, iN)     # calculate the energy
        Mag = mag(pArr)        # calculate the magnetisation
        if i % 5 == 0:
            mapPlot(ax,fig,pArr,i,iN,images)

images=[]
fig, ax = plt.subplots(figsize=(12,7))
runTemp(1.00,32,images,fig,ax)
imageio.mimsave('./test.gif', images, fps=10)
Image(open('test.gif','rb').read())

Play around with the tempature. How does the Ising model look for different tempatures?

Do you visualize a phase transition? (if you are impatient, move on!0

#### Step 1.1.2 Modelling the Phase transition

Now, what we would like to do for each of our simulations is compute a bunch of quantities about the matter. In particular, we would like to compute the following quantities for the Ising model:

 * Average Energy $\langle E \rangle$ averaged over all the cells of the Ising model
 * Average Magnetization $\langle M \rangle$ averaged over all the cells of the Ising model
 * Specific Heat $C=\frac{\langle E^{2}\rangle - \langle E\rangle^{2}}{T^{2}}$ where $E$ is the cell energy averaged over all the cells of the Ising model
 * Magnetic Susceptibility $\chi=\frac{\langle M^{2}\rangle - \langle M\rangle^{2}}{T}$ where $M$ is the magnetization averaged over all of the cells in the Ising model

Now the importance of this is that we would like to scan the temperature and observe a phase transition. The specific heat,  in particular, has an infinite discontinuity in the precence of a phase trasition.

In the below modify the function to output thse variable.

In [ ]:
def runTemp(iT,iN,images,fig,ax,eqSteps=500,mcSteps=500):
    pArr = initialize(iN)         # initialise
    #initial variables? 
    beta=1.0/iT 
    for i in range(eqSteps):         # equilibrate
        update_rand(pArr, iN, beta)   
    
    for i in range(mcSteps):
        update_rand(pArr, iN, beta)           
        Ene = hamiltonian(pArr, iN)     # calculate the energy
        Mag = mag(pArr)        # calculate the magnetisation

        #perhaps some code here to get below
        
    #compute the values for E,M,C,X here
    E = 
    M = 
    C = 
    X = 
    return E,M,C,X


Now we have computed all of these things, lets go ahead and scan the temperature to see what is going on. Make a scan of temperature from $k_{b}T = 1.5$ to $3.3$.

As you scan along plot the different values for E,M,C,$\chi$ Where do you see the phase transition? What characterizes this phase transition?

In [ ]:
N=#Something (your choice, choose wisely)
nt =#Number of temperature points
T  = #np.linspace(1.5-ish,3.3-ish,nt) #again pick
E,M,C,X = np.zeros(nt), np.zeros(nt), np.zeros(nt), np.zeros(nt)
for temp in tqdm (range (nt), desc="Loading..."):
    E[temp],M[temp],C[temp],X[temp] = runTemp(T[temp],N,images,fig,ax,eqSteps=500,mcSteps=500)


Now, given our above simulation. Go ahead and plot the the various parameters. What do you see? (It might be worth checking this with resources)

In [ ]:
f = plt.figure(figsize=(18, 10)); #  

sp =  f.add_subplot(2, 2, 1 );
plt.scatter(T, E, s=50, marker='o', color='IndianRed')
plt.xlabel("Temperature (T)", fontsize=20);
plt.ylabel("Energy ", fontsize=20);         plt.axis('tight');


sp =  f.add_subplot(2, 2, 2 );
#Plot magnetization

sp =  f.add_subplot(2, 2, 3 );
#Plot specific heat

sp =  f.add_subplot(2, 2, 4 );
#Plot magnetic susceptibility


From the above, it should be clear there is a critical temperature for the Ising model. Its in weird units, but we could easily conver this to a normal temperature. The other thing you can see wright away is how this material magnetizes and how it can change as a function of the different temperature regimes.

The full 2D Ising model has also been solved analytically, with a complete solution coming in the past 10 years. This was the motivation for the Fields medal in 2022 for Hugo Duminil-Copin at the University of Geneva. That being said the properties of criticallity and a simplfied solution have been known since 1950.

Can you validate that you get the right critical tempature from analytic calcualtions ($T_{c} = 2/\log(1+\sqrt{2})$)

## Step 1.2:  Constructing Ising Model Simulation

Now that we have generated an Ising model, we want to build an optimized scheme to run the Ising model many times so that we can train a neural network to understand the critical temperature of the ising model.

#### 1.2.1 Training an NN

For this part of the lab, we would like to generate Ising Monte Carlo Simulations where we randomly sample many configurations, and we the evolve the configuration at a specific temperature and then save it. Practically means we need to make simulated events where in each event, we

Randomly sample a config
Evolve that config  𝑁 steps ( 𝑁≈500 )
Save the evolved configuration, magnetization and temperature
Repeat the above nsim times and write this all to disk
Once, we have done that then we can use the datasets have generated to make a neural network that takes as input the random configuration and outputs the temperature.

So now lets make a class that generates Ising configurations. We can use use the previous functions. The important function in our class is the simulate_save, this will write a file with all of our Ising simulations.

A class setup is below for you to fill out.



In [ ]:
import h5py 

class Ising():
    
    def __init__(self, iN, Temp):
        self.N   = iN
        self.T   = Temp
        self.arr = self.initialize()
        self.steps = 300
        #History over simulatinp
        self.E   = []
        self.M   = []
        self.C   = []
        self.X   = []
        self.nsim = 1000
        
    def initialize(self):   
        #use previous function
        return state
    
    def simulate(self):
        beta = 1./self.T
        for i in range(self.steps):
            update_rand(self.arr, self.N, beta)           
            Ene = hamiltonian(self.arr, N)
            Mag = mag(self.arr)
            #Now save energy magnetization 
            self.E.append(Ene)
            self.M.append(Mag)
            #Now COMPUTE specific Heat and Magnetic suscpetilibity
            #HINT, consider what the meaning of RMS of Energy and Magnetization are
            #Perhaps consider a sliding window over the last hundred steps
            pC  = #code here
            pX  = #code here
            self.C.append(pC)
            self.X.append(pX)

    def simulate_save(self,pre=''):
        h5f  = h5py.File((pre)+'data_'+str(self.T)+'.h5', 'a')
        data = np.array([])#np.empty((1,self.N,self.N), int)
        mags = np.array([])
        TM1  = 1./self.T
        for n in range(self.nsim):
            if n % 25 == 0:
                print("sim",n)
            self.initialize()
            ## Add code to run simulate the ising model nsteps
            ## update self.arr  

            #for each simulation we want to save the magnetization and the array      
            pMag = mag(self.arr)
            data.append(self.arr)
            mags.append(pMag)
        #now we write the output array into a dataset
        data = np.array(data)
        data = np.reshape(data,(self.nsim,self.N,self.N))
        h5f.create_dataset('data', data=data)
        h5f.create_dataset('mag' , data=mags)
        h5f.close()
                    
    def lastAvg(self):
        avgE = np.mean(self.E[500:-1])
        avgM = np.mean(self.M[500:-1])
        avgC = np.std(self.E[500:-1])
        avgX = np.std(self.M[500:-1])
        return avgE,avgM,avgC,avgX
        
    def plotEvol(self):
        ts = range(len(self.E))
        f = plt.figure(figsize=(18, 10)); #  
        sp =  f.add_subplot(2, 2, 1 );
        plt.scatter(ts, self.E, s=50, marker='o', color='IndianRed')
        plt.xlabel("step", fontsize=20);
        plt.ylabel("Energy ", fontsize=20);         plt.axis('tight');

        #PLOT THE MAGNETIZATION, SPECIFIC HEAT AND SUSCEPTIBILITY
        
test = Ising(64,3.4)
test.simulate()
test.plotEvol()

Alright now that we have a class to run our Ising model and save things to disk we can go ahead and generate some samples following the code below, lets first generate a test sample so that we understand how to train the neural network. For this, lets just generate 10 test samples for each point, this shouldn't take too long, and will allow us to setup the neural network.

Also, to make our ising model manageable, lets use a 32x32 Ising model. Note, feel free to change this! Second Note, this box will take a bit of time to run, its set up to do test run with 10 first, but at some point you should switch to 500.  

After you test it with 10, you may want to do the run with 500 by following the optional HPC section below; this will make the calculation run much faster.

In [ ]:
import os
nt=20
T = np.round(np.linspace(1.53, 3.28, nt),2)
print(T)

def simulate_single_T(temp, T=T):
    ''' temp = index to run in array T of temperatures'''
    #Some hacky code to clean up things
    filename='data_'+str(T[temp])+'.h5'
    try:
        os.remove(filename)
    except OSError:
        pass
    test = Ising(32, T[temp])
    test.nsim=10#500 =>
    test.simulate_save()


for temp in tqdm (range (nt), desc="Loading..."):
    simulate_single_T(temp)

f = h5py.File('data_1.53.h5', 'r')
list(f.keys())
f['data'].shape
!ls

##### 1.2.1.1 (OPTIONAL) HPC-style Speedup

Note: you can toggle showing/hiding this section by clicking on the arrow to the left of this section heading

There are *many* different ways to utilize High Performance Computing (HPC) style resources.  The choice often depends on the workflow and the resources available.  We will follow the steps below to introduce you to a variety of methods, so you will be prepared for most resources you are likely to encounter in your future academic career.  Each of these runs a parallelization of our calculation

1. Running *interactively* using multiprocessing (on your laptop, then on the HPC style cluster via SLURM)
2. Running *batch* jobs as separate SLURM jobs
3. Running *batch* jobs as a SLURM job array
4. Running *batch* job as a single multi-core SLURM job

The first step is to run the above loop over temperatures for a small number of temperatures for a small value of `test.nsim` to make sure the code is working properly.

###### 1.2.1.1.1 Interactive with multiprocessing (on your laptop or HPC cluster)

This was written assuming you would walk through this section on your laptop first and then repeat on the cloud cluster.  But you could alternatively start by just running this on the cloud cluster.  To do so, please see the instructions at the very end of this section for how to run this on the cluster, then return to this point.


For our purposes, we will call the smallest hardware unit which can perform an independent calculation a *core*.  The computer on which you are running this notebook (laptop, colab, ...) likely has more than one core.  Here we will adapt the above code to take advantage of them, a technique known as `parallelization`.  An important concept is that code must be specially written to actually run faster on a multi-core machine.  Code which was not written for parallelization is called `serial` code.  Running serial code on a multi-core machine will *not* result in a speedup; instead, most of the cores will sit idle.

To speed up our calculation in this way, we must first *identify* which portions of our workflow may be performed independently, aka "in parallel".  If you had several people willing to help you perform this calculation, how could you split it up so that each person could perform an independent task, resulting in the work being completed faster?  How much would these people have to talk to each other or share resources, and when?

This raises another crucial concept for parallelization: `contention`.  If two people write to the same chalkboard at the same time, it will, in the best case, slow down the calculation, and in the worst case, make the results incorrect.  The same may occurs in computers if two independent workers try to access the same file, location in memory, GPU, or other resource.


<details>
<summary>More Detail: contention (click arrow to show/hide)</summary>

> One extremely important concept to be aware of when implementing parallelization is that of contention. Contention is when multiple would-be independent tasks are trying to use the same resource (processor, memory, I/O, network, GPU, etc.) at the same time. In the worst case, contention can cause unpredictable, incorrect behavior.
> 
> Consider the example that you and your friend are working out different math problems on the same whiteboard. If, unbeknownst to you, your friend erases intermediate numbers you wrote and replaces them with their own numbers, you will get an incorrect result if you then use those numbers later in your calculation. And worse yet, you may not know it! In this analogy, you can think of the whiteboard as shared memory and you and your friend each as processors. We will show you modern high-level tools which often have protections against such catastophic failures, but always keep contention in mind because sometimes you need to know when to invoke protections manually. One example is that it is easy to make the mistake of 2 different processes writing to the same file; protections such as file locks exist, but you need to know when to use them. Also, if you perform more manual, lower-level implementations of parallelization, you must consider contention.
> 
> Even with protections against such catastrophic contention failures in place, you still need to consider contention from a standpoint of performance (e.g. speed or efficient resource usage). Let's say there is only one marker for you and your friend to share. While this will not jeopordize the correctness of your reults, it will slow you down. Even more so if there is one marker for 10 students to share; this is known as a bottleneck. Considering contention and possible bottlenecks when deciding on implementation details can minimize and possibly even eliminate performance hits due to contention.
> 
> One example you can look up on your own is "race conditions".

</details>





The above workflow contains steps which *could* be run simultaneously or even asynchronously in any order; can you spot it?  
Hint: if looking at code, a good place to start is examining loops.

Notice that the calculation for each temperature depends in no way on any of the calculations at other temperatures.  Thus, each iteration of the above loop over temperatures could be run independently, in parallel, then the results gathered together in the end.

`Concept Question:`  Could the loop over spin flips within `Ising.simulate` be parallelized by running each loop iteration independently/simultaneously?

`Answer:`  No.  The Metropolis Algorithm evaluates the *change* in energy between successive states.  Thus the iterations must be run sequentially, aka serially.  (The "chain" in Markov Chain Monte Carlo).

`Concept Question:` Often, when simulating a lattice, one is trying to calculate results for a lattice of infinite size (but of course you must simulate a finite sized lattice).  Therefore, a common task when simulating a lattice is to check for convergence of calculated quantities with respect to the size of the lattice, repeating the calculation for increasing lattice sizes.  Can you use further parallelization (beyond parallelization over temperature) to speed up this type of study?

`Answer:`  Yes, you can run different lattice sizes in parallel, since these are completely independent calculations.

The next step is to construct a function which may be run independently for each (input) temperature.  Beware of contention!  In this case, such a function already exists: `simulate_single_T`.  Note that writing the output to a file based on the index of the temperature avoids contention of different workers for different temperatures trying to write to the same file.

Next, run the next code cell to see how many cores you have available to you on this machine.  Note: if you are running on a cluster, this may be less than the number of cores on the machine; the `salloc` statement tells the resource manager to allot you a certain number of cores.

In [ ]:
import os
num_cores = os.cpu_count()
print(f"You have {num_cores} cores available to you on this machine.")

In this exercise, we will have each core work independently on a task using its own separate memory.  This is known as *multi-processing*, and each independent worker is called a *process*.  A related strategy is *multi-threading*, where the workers can share memory, but we will not discuss that here.  There are *many* tools which help with writting parallel code: dask, pathos, ipyparallel, multiprocessing, and concurrent.futures modules, to name only a few.  Some of the built-in python parallelization modules require special care when running inside a Jupyter notebook, so we will use dask, for simplicity.  If you are on your laptop, run the cell below to install it (you may skip this if running on the cluster).

In [ ]:
# If running on your laptop, install
# (This is already installed on colab and the cloud cluster)
%pip install "dask[distributed]"

Run the next cell, which simulates a parallel calculation.  Here, `fake_calculation` just sleeps for a specified length of time to simulate an expensive calculation.  First, `fake_calculation` is run mulitiple times serially, then the same number of times in parallel.  You should observe that the factor in speedup is roughly equal to the number of cores used.  Note how the speedup here comes only from running separate instances of `fake_calculation` (separate processes) *simultaneously*; each process runs the same fixed amount of time.

`Syntax`: To define what the processes will execute, we create lists of the different input arguments for the different processes.  The processes are started by calling a `map` function of the form `client.map(function_name, list_arg_1, list_arg_2, list_arg_3, ...)`.  This means that the `i`-th process will run `function_name(list_arg_1[i], list_arg_2[i], list_arg_3[i])`.  Many tools use this type of syntax.

<details>
<summary>More Detail (click arrow to show/hide)</summary>

* We cheat a little in calculating the speedup here in that we do not include the overhead in setting up the dask cluster for parallelization.  We do this because the cluster can be set up once and reused for many subsequent parallel calculations.

* A Dask "cluster" is different from a Linux or HPC cluster.  A Linux cluster is a set of machines (see HPC intro section).  A Dask cluster is a set of workers and may be set up on your laptop, colab, or on machines within a HPC Linux cluster.  

</details>

In [ ]:
# Example: Simulated Parallel Calculation
import os, time, logging
from dask.distributed import Client, LocalCluster
logging.getLogger("distributed").setLevel(logging.WARNING)

# Set up Dask cluster (set of independent workers)
cluster = LocalCluster()  # Create a cluster of workers using all cores
client = Client(cluster)  # Connect to the cluster

# Define the function to run in parallel
def fake_calculation(call_number):
    '''This function is a slacker. It pretends to do a long calculation, but really just sleeps'''
    process_id = os.getpid()  # a unique identifier for this process
    report = f"Function call ID {call_number} started  at {time.strftime('%H:%M:%S')} with Process ID: {process_id}"
    time.sleep(5)  # sleep for 5 seconds to simulate a long calculation
    return report

# Number of executions
num_cores = os.cpu_count()
num_executions = min(4, num_cores)
print(f"You have {num_cores} cores available to you on this machine. We will use {num_executions} of them.")

# Serial execution
print(f"\nRunning {num_executions} calls to fake_calculation in serial ...")
start_time = time.time()
for i in range(num_executions):
    result = fake_calculation(i)
    print(result)
total_seconds_serial = time.time() - start_time
print(f"During the serial execution, the clock on the wall advanced {total_seconds_serial:.2f} seconds")

# Parallel execution with Dask
print(f"\nRunning {num_executions} calls to fake_calculation in parallel ...")
start_time = time.time()
# Submit tasks to the Dask cluster
input_list = range(num_executions)  # list has one entry for each process
futures = client.map(fake_calculation, input_list)
# Gather results
results_parallel = client.gather(futures)
# Print details
for report in results_parallel:
    print(report)
total_seconds_parallel = time.time() - start_time
print(f"During the parallel execution, the clock on the wall advanced {total_seconds_parallel:.2f} seconds")

# Print speedup
print(f"Using {num_executions} cores simultaneously, the parallel execution ran {total_seconds_serial / total_seconds_parallel:.2f} times faster (measured by the wall clock)")

# Close the Dask client & cluster
client.close()
cluster.close()

The main steps of 1) setting up a set of workers 2) mapping a function / inputs to workers 3) gathering results from workers are common in the various multiprocessing tools, though the syntax and details may vary.

Getting back to the main task, we are ready to generate the training set in parallel.  To do so, run the next cell, which adapts the example above to the training set generation.  Note: the original serial loop calling `simulate_single_T` with different input values (aka arguments) is now replaced with a `map` call with lists of different input values to spawn parallel processes.  You can set test.nsim small to test, but then set it to 500.

<details>
<summary>More Detail (click arrow to show/hide)</summary>

In this notebook, we set up and destroy the dask cluster in each cell.  This is in case you run cells in isolation or out of order.  In a more realistic workflow, you would set up a dask cluster once (eg in an early cell), then leave it running for subsequent parallel calculations in other cells, then destroy the dask cluster when you are all finished.

</details>

In [ ]:
import os, time, logging
from dask.distributed import Client, LocalCluster

logging.getLogger("distributed").setLevel(logging.WARNING)

# Start with the same code from the original cell above
nt=20
T = np.round(np.linspace(1.53, 3.28, nt),2)
print(T)

def simulate_single_T(temp, T=T):
    ''' temp = index to run in array T of temperatures'''
    #Some hacky code to clean up things
    filename='data_'+str(T[temp])+'.h5'
    try:
        os.remove(filename)
    except OSError:
        pass
    test = Ising(32, T[temp])
    test.nsim=500#500 =>
    test.simulate_save()

# Now replace the loop w/ new code to run in parallel ...
list_temp = range(nt)  # Make list of inputs to define what each process (worker) will run
list_T = [T]*nt        # Make list of other input: just repeat this one (same for each worker)
# Run the calculation in parallel using Dask (see earlier example)
cluster = LocalCluster() ; client = Client(cluster)
futures = client.map(simulate_single_T, list_temp, list_T)
client.gather(futures)
client.close() ; cluster.close()

# Continue with the rest of the original (serial) cell here
f = h5py.File('data_1.53.h5', 'r')
list(f.keys())
f['data'].shape
!ls

Congratualtions!  You've now run the calculation in parallel!  Besides saving time with the above calculation, you can also more quickly explore: increasing the lattice size, running longer simulations, running additional temperatures.

If you completed this on your laptop or colab, please go back and repeat this on the cloud cluster.  This will greatly help you in your future career in that it will teach you how to access additional computational power beyond your laptop.  These skills are not limited to cloud services; many institutions (including MIT) have their own free in-house cluster(s).

If you wish to run this on the cloud or local HPC style Linux cluster, expand the `More Details` section for some helpful background and the `HPC style cluster instructions` section below for specific instructions on how to connect.  Then return to the top of this section and re-run it on the cluster!


<details>
<summary>More Detail: HPC Clusters (click arrow to show/hide)</summary>

##### TL;DR
* HPC resources achieve speedup via parallelization.  Thus, earlier comments about speedup & contention apply to HPC clusters.
* A HPC style cluster consists of many interconnected nodes (machines) which each have many cores.  Some have special resources such as GPUs attached.
* Login nodes are only for lightweight use; submit heavy tasks (jobs) to compute nodes via SLURM (the scheduler)
* Interactive jobs require user input; batch jobs run without user input after waiting in a queue

##### Intro: 

For a given set of computational resoures (e.g. your laptop) complexity of physics problems one can tackle computationally may be limited by:
* the time required to perform the calculation
* the memory required
* the storage (disk) space required

You can modify your code to address these concerns, but at some point, to tackle more complex problems, you will need to expand your computational resources.  Common advanced resources include multicore CPU machines, accelerators (such as GPUs), high performance (HPC) or high throughput (HT) style linux clusters, and grid and cloud computing.  These are overlapping areas in the Venn diagram of computing.

##### Detailed info:
If your laptop has more than one core, you saw parallel computing on your laptop above.  Computational tasks can also be distributed across multiple cores on multiple machines.  There are different styles or architectures of such collections of machines; this all falls under the general heading of *distributed computing*.  In this course, we will use what is often called an HPC-style linux cluster which is hosted on Cloud resources.  Many univerities also host local HPC style linux clusters.

A HPC style cluster, consists of many different machines connected via a network; each machine is called a *node*, and each node contains many *cores*.  Some nodes may contain specialized hardware such as GPUs.  Since many different users share these resources, the use of the resources is orchestrated by a scheduler or resource manager; in our case we will use SLURM.  The scheduler is one level of handling contention.

Clusters typically have login nodes (aka head nodes) and compute nodes (aka backend nodes or worker nodes).  When you log into a cluster, you will land on a login node; login nodes are *not* meant for heavy calculation - they are meant for lightweight tasks and submitting heavy calculations, known as *jobs* to the scheduler.  The jobs submitted to the scheduler wait in a queue and then the scheduler launches the jobs on the compute nodes.  In general, a user should only interact with compute nodes through the scheduler.  One may submit *interactive* or *batch* jobs to the sheduler.  Batch jobs run on compute nodes without any input from the user (typically at some unknown later time), wheras *interactive* jobs typically run soon (hopefully immediately) after they are submitted and also run on compute nodes but with user input.

Batch jobs are typically submitted to the scheduler via a script, commonly referred to as a *job script* or *batch script*, which will tell the scheduler what resources our job will use (# of cores, GPUs, memory, time, etc) and what commands to run.  Scripts can get more complicated than this but let's stick with the basics for now.

Each node has its own local filesystem (on each machine) as well as access to a common shared filesystem which is accessible by all nodes.  Your `home` directory lives on this shared filesystem and so is accessible on all nodes.  Aside: this is different from other architectures such as some grid computing resources which do not have a user-writable shared directory acessible by each node, such as the Open Science Grid.  But the main concepts and tecniques you will learn are transferrable to those architectures as well with some modification.

Note: there are other types of systems & schedulers including Kubernetes (K8), HTCondor, Google Batch / Cloud Run, etc, but the main concepts and core skill set is transferrable.

</details>










<details>
<summary>Instructions for running on cloud HPC Cluster (click arrow to show/hide)</summary>

##### Initial Setup

If you have not yet completed the Initial Setup once, you must do that first.  Click the arrow below for instructions.  You should only have to do this *once* this semester (unless you get a new laptop).

<details>
<summary>Initial One-Time Setup Instructions (click arrow to show/hide)</summary>

1.  Install Google Cloud CLI: [text](https://cloud.google.com/sdk/docs/install)
  - Leave the default options for the various steps (install bundled python, run `gcloud init`, etc)
  - Eventually, a browser window will pop up asking you to sign into Google giving Google Cloud SDK access to your Google Account; do this.

2.  When you are finished with the browser window that pops up, close it *AND RETURN to the terminal window* which was open before the browser, and complete initialization there.  
  - *IF* there is no longer a terminal window open, run 
  open the Google Cloud CLI link which should have been installed on your desktop and then run `gcloud init` when that opens.  Choose the following:
  - Set your default `cloud project` to: `mit-teaching-project`
  - Set your default `Compute Region and Zone` to: `us-central1-b`

3.  It may be a good idea to restart your computer now; if you do not, keep in mind to try restarting if you run into any future problems.

</details>


##### Run Jupyter on a *CPU compute node*

We will launch Jupyter notebooks in a way which will give you 2 great tools:

* manually starting a Jupyter server (on a remote machine in a custom enviornment)

* ssh port forwarding (useful even beyond Jupyter notebooks)

<details>
<summary>More Detail (click arrow to show/hide)</summary>

Some clusters may have JupyterHUB set up.  This is a tool which can automatically perform these tasks for you, but it's still good to know how to do this yourself.  JupyterHUB usually provides predefined options; with the techniques we will show you, you can request custom resources.

</details>

Following these directions, your jupyter lab session (notebooks, terminal) will have access to all of the resources (cores, memory, ...) specified in your SLURM request (the `salloc` statement).  This setup will involve you opening/using 2 different terminal windows on your laptop; we will label them `Terminal A` and `Terminal B`.

Note: if you are on a Windows machine, then by "terminal" below, we mean the window opened when you double-click on the `Google Cloud SDK Shell` icon on your Desktopm, or go to Start (Window button) and type in `Google Cloud SDK Shell`.  On a Mac, open the terminal by going to Launchpad, typing `Terminal` in Spotlight Search (CMD+Space), or in Finder open `Applications` then `Utilities` folders then click on Terminal.

In the terminal on your laptop (we will call this terminal window `Terminal A`), run the following comand (you may be prompted to enter your ssh key password)

```bash
gcloud compute ssh hpc-slurm-login-001 --tunnel-through-iap
```

<details>
<summary>If this is your first time connecting ... (click arrow to show/hide)</summary>

you may see a warning(s) like below:

```bash
WARNING: The private SSH key file for gcloud does not exist.
WARNING: The public SSH key file for gcloud does not exist.
WARNING: The PuTTY PPK SSH key file for gcloud does not exist.
WARNING: You do not have an SSH key for gcloud.
WARNING: SSH keygen will be executed to generate a key.
This tool needs to create the directory [C:\Users\muser2\.ssh] before being able to generate SSH keys.

Do you want to continue (Y/n)?
```

and/or perhaps like

```bash
The host key is not cached for this server:

compute.783492966651981141 (port 22)

You have no guarantee that the server is the computer you think it is.

The server's ssh-ed25519 key fingerprint is:

ssh-ed25519 255 SHA256:QHZjTMoUqO2yxfSn8UtkVcTpozMgvPQgfuTjmVAvQy0

If you trust this host, press "Accept" to add the key to PuTTY's cache and carry on connecting.
```

but with the numbers/characters different.  This is OK and expected.  Please continue.

</details>




<details>
<summary>If that did not work ... (click arrow to show/hide)</summary>

* If you have MIT Kerberos Ticket Manager installed, when you run the above, the connection may seem to hang (freeze).  This typically means a login window for MIT Kerberos Ticket Manager is open somewhere (often instigated by the above command).  Look to see if Kerberos Ticket Manager is open behind the terminal or other windows, or in another desktop; if so, hit cancel to close it.  Then the connection should proceede.

* If you already have Google Cloud CLI installed and use it for other projects, you will have to add these aruments to the above line to specify project name and region/zone this this course's project: `--project=mit-teaching-project --zone=us-central1-b`

</details>

This should connect you to a login node (you can confirm by typing `hostname` followed by return and it should print `hpc-slurm-login-001`).  Then run the following commands in that terminal (Note: Specific exercises instruct you to replace this `salloc` statement with a different one to request more resources)

```bash
salloc -N 1 -c 8
```

This requests `8` cores on 1 node.  You may have to wait for a while after the `salloc` statement for a node to become free or a new virtual node to be created.  Once you are notified your resources are ready and get a terminal prompt again, run

```bash
hostname
```

Take note of the output printed by the `hostname` command; this is the name of the compute node on which your notebook will run.  Then run the following

```bash
source /opt/venv/bin/activate
jupyter lab --no-browser --port 8080
```

The first line will activate the python virtual environment in which we have installed the necessary software.  Virtual environments are a good practice for isolating/managing packages.  The second line will start your jupyter lab server and print a lot of text to the screen, including lines like `Or copy and paste one of these URLs:` followed by a url which looks like `http://localhost:8080/lab?token=[...]` (where the `...` is a long string of letters & numbers).  Take note of this URL.

Now open a new terminal window on your laptop (we will call this `Terminal B`) and run the following command in it

```bash
gcloud compute ssh hpc-computecpu-0 --tunnel-through-iap -- -L 8080:localhost:8080
```

where instead of `hpc-computecpu-0`, write the name of the compute node (the name returned by the `hostname` command in the earlier step above).  This should just log in to the cluster and look like a regular ssh session; when it does that, just leave it open.

Now paste the URL mentioned above into your web browser on your laptop.  This should open Jupyter Lab in your web browser.  Commands you run in this Jupyter session are actually being run *on the cluster node*.

If you want to run a particular notebook on this cluster, simply drag and drop the notebook file into the Files sidebar of the Jupyter window in your browser.  (If your notebook is on colab, you will have to download your notebook, then drag the downloaded file into Jupyter Lab).

<details>
<summary>Troubleshooting (click arrow to show/hide)</summary>
  
* If you get an error that the port is not available, then please start over again substituting a different number for `8080` above.

</details>

Now return to the beginning of this section to start the exercise in this cloud cluster session.

</details>




<!-- <details>
<summary>Instructions for running on Engaging HPC Cluster (click arrow to show/hide)</summary>

For more information and troubleshooting, see: https://orcd-docs.mit.edu 
##### Initial Setup

If you have not yet completed the Initial Setup once, you must do that first.  Click the arrow below for instructions.  You should only have to do this *once* this semester (unless you get a new laptop).

<details>
<summary>Initial One-Time Setup Instructions (click arrow to show/hide)</summary>

1. Connect to the cluster by running in a terminal: `ssh <username>@orcd-login001.mit.edu` where you should replace `<username>` with your kerberos id (beginning part of your mit email address).

To open terminal: if you are on a Windows machine, go to Start (Window button) and type in `Terminal`.  On a Mac, open the terminal by going to Launchpad, typing `Terminal` in Spotlight Search (CMD+Space), or in Finder open `Applications` then `Utilities` folders then click on Terminal.


2. Make a private directory for this class by running the following commands

```bash
mkdir ~/private_816
chmod 700 ~/private_816
```

3. Install sofware by running

```bash

python3 -m venv ~/venv816
source ~/venv816/bin/activate
python3 -m pip install --upgrade pip
python3 -m pip install numpy scipy matplotlib tqdm imageio h5py ipykernel jupyterlab jupyter-server-proxy dask[distributed] pathos joblib ipyparallel ipywidgets torch torchvision torchaudio torchmetrics pyright python-language-server python-lsp-server

```
4.  You can log out now by running `exit` followed by Enter/Return

</details>


##### Run Jupyter on a *CPU compute node*

We will launch Jupyter notebooks in a way which will give you 2 great tools:

* manually starting a Jupyter server (on a remote machine in a custom enviornment)

* ssh port forwarding (useful even beyond Jupyter notebooks)

<details>
<summary>More Detail (click arrow to show/hide)</summary>

Some clusters may have JupyterHUB set up.  This is a tool which can automatically perform these tasks for you, but it's still good to know how to do this yourself.  JupyterHUB usually provides predefined options; with the techniques we will show you, you can request custom resources.

</details>

Following these directions, your jupyter lab session (notebooks, terminal) will have access to all of the resources (cores, memory, ...) specified in your SLURM request (the `salloc` statement).  This setup will involve you opening/using 2 different terminal windows on your laptop; we will label them `Terminal A` and `Terminal B`.

In the terminal on your laptop (we will call this terminal window `Terminal A`), run the following command but replace `<username>` with your kerberos id (beginning part of your mit email address):

```bash
ssh <username>@orcd-login001.mit.edu
```

This should connect you to a login node (you can confirm by typing `hostname` followed by return and it should print `orcd-login001`).  Then run the following command in that terminal

```bash
salloc -N 1 -p mit_quicktest -c 10
```

This requests `10` cores on one node.  You may have to wait for a while after the `salloc` statement for the resources (cores on a node) to become available.  *Note:* This allocation will only last for 15 minutes!  If you want to work longer without needing to redo the steps to restablish the connection, you can replace `mit_quicktest` in the line above with `mit_normal`, but you may have to wait longer (or reduce the number of cores in the request, `-c <# cores>`).  Once a new prompt shows up, run

```bash
hostname
```

Take note of the output printed by the `hostname` command; this is the name of the compute node on which your notebook will run.  Then run the following

```bash
cd ~/private_816
source ~/venv816/bin/activate
jupyter lab --no-browser --ip=0.0.0.0  --port 8080
```

The first line will activate the python virtual environment in which you have installed the necessary software.  Virtual environments are a good practice for isolating/managing packages.  The second line will start your jupyter lab server and print a lot of text to the screen, including lines like `Or copy and paste one of these URLs:` followed by a url which looks like ` http://127.0.0.1:8080/lab?token=[...]` (where the `...` is a long string of letters & numbers).  Take note of this URL.

Now open a new terminal window on your laptop (we will call this `Terminal B`) and run the following command in it

```bash
ssh -L 8080:nodename:8080 <username>@orcd-login001.mit.edu
```

where instead of `nodename`, write the name of the compute node (the name returned by the `hostname` command in the earlier step above).

Now paste the URL mentioned above into your web browser on your laptop.  This should open Jupyter Lab in your web browser.  Commands you run in this Jupyter session are actually being run *on the cluster node*.

If you want to run a particular notebook on this cluster, simply drag and drop the notebook file into the Files sidebar of the Jupyter window in your browser.  (If your notebook is on colab, you will have to download your notebook, then drag the downloaded file into Jupyter Lab).

<details>
<summary>Troubleshooting (click arrow to show/hide)</summary>
  
* If you get an error that the port is not available, then please start over again substituting a different number for `8080` above.

</details>

*Note:* If running on Engaging, you will need to make the following change to **ALL** cells below where these appear

* change the line `source /opt/venv/bin/activate` to `source ~/venv816/bin/activate`
* change all references to the partition `pcpu` to `mit_quicktest` or `mit_normal`

Now return to the beginning of this section to start the exercise in this cloud cluster session.

</details>



<details>
<summary>Instructions for running on subMIT Cluster (click arrow to show/hide)</summary>

For more information and troubleshooting, see: https://submit.mit.edu 
##### Initial Setup

If you have not yet completed the Initial Setup once, you must do that first.  Click the arrow below for instructions.  You should only have to do this *once* this semester (unless you get a new laptop).

<details>
<summary>Initial One-Time Setup Instructions (click arrow to show/hide)</summary>

1. Follow instructions in https://submit.mit.edu/submit-users-guide/ to create an account.

1. Connect to the cluster by running in a terminal: `ssh <username>@submit.mit.edu` where you should replace `<username>` with your kerberos id (beginning part of your mit email address).

To open terminal: if you are on a Windows machine, go to Start (Window button) and type in `Terminal`.  On a Mac, open the terminal by going to Launchpad, typing `Terminal` in Spotlight Search (CMD+Space), or in Finder open `Applications` then `Utilities` folders then click on Terminal.

To open terminal: if you are on a Windows machine, go to Start (Window button) and type in `Terminal`.  On a Mac, open the terminal by going to Launchpad, typing `Terminal` in Spotlight Search (CMD+Space), or in Finder open `Applications` then `Utilities` folders then click on Terminal.

2. Make a private directory for this class by running the following commands

```bash
mkdir ~/private_816
chmod 700 ~/private_816
```

3. Install sofware by running

```bash

python3 -m venv /work/submit/${USER}/venv816
source /work/submit/${USER}/venv816/bin/activate
python3 -m pip install --upgrade pip
python3 -m pip install numpy scipy matplotlib tqdm imageio h5py ipykernel jupyterlab jupyter-server-proxy dask[distributed] pathos joblib ipyparallel ipywidgets torch torchvision torchaudio torchmetrics pyright python-language-server python-lsp-server

```
4.  You can log out now by running `exit` followed by Enter/Return

</details>


##### Run Jupyter on a *CPU compute node*

We will launch Jupyter notebooks in a way which will give you 2 great tools:

* manually starting a Jupyter server (on a remote machine in a custom enviornment)

* ssh port forwarding (useful even beyond Jupyter notebooks)

<details>
<summary>More Detail (click arrow to show/hide)</summary>

Some clusters may have JupyterHUB set up.  This is a tool which can automatically perform these tasks for you, but it's still good to know how to do this yourself.  JupyterHUB usually provides predefined options; with the techniques we will show you, you can request custom resources.

</details>

Following these directions, your jupyter lab session (notebooks, terminal) will have access to all of the resources (cores, memory, ...) specified in your SLURM request (the `salloc` statement).  This setup will involve you opening/using 2 different terminal windows on your laptop; we will label them `Terminal A` and `Terminal B`.

In the terminal on your laptop (we will call this terminal window `Terminal A`), run the following command but replace `<username>` with your kerberos id (beginning part of your mit email address):

```bash
ssh <username>@submit.mit.edu
```

This should connect you to a login node (you can confirm by typing `hostname` followed by return and it should print `orcd-login001`).  Then run the following command in that terminal

```bash
salloc -c 4
```

This requests `4` cores on one node.  You may have to wait for a while after the `salloc` statement for the resources (cores on a node) to become available.  Once a new prompt shows up, run

```bash
hostname
```

Take note of the output printed by the `hostname` command; this is the name of the compute node on which your notebook will run.  Then run the following

```bash
cd ~/private_816
source /work/submit/${USER}/venv816/bin/activate
jupyter lab --no-browser --port 8080
```

The first line will activate the python virtual environment in which you have installed the necessary software.  Virtual environments are a good practice for isolating/managing packages.  The second line will start your jupyter lab server and print a lot of text to the screen, including lines like `Or copy and paste one of these URLs:` followed by a url which looks like ` http://127.0.0.1:8080/lab?token=[...]` (where the `...` is a long string of letters & numbers).  Take note of this URL.

Now open a new terminal window on your laptop (we will call this `Terminal B`) and run the following command in it

```bash
ssh -L 8080:localhost:8080 -J <username>@submit.mit.edu <username>@hostname.mit.edu
```

where instead of `nodename`, write the name of the compute node (the name returned by the `hostname` command in the earlier step above).

Now paste the URL mentioned above into your web browser on your laptop.  This should open Jupyter Lab in your web browser.  Commands you run in this Jupyter session are actually being run *on the cluster node*.

If you want to run a particular notebook on this cluster, simply drag and drop the notebook file into the Files sidebar of the Jupyter window in your browser.  (If your notebook is on colab, you will have to download your notebook, then drag the downloaded file into Jupyter Lab).

<details>
<summary>Troubleshooting (click arrow to show/hide)</summary>
  
* If you get an error that the port is not available, then please start over again substituting a different number for `8080` above.

* If this still does not work, you can get a mulit-cpu Jupyter session via JupyterHub at http://submit.mit.edu/jupyter  In this case, you will have to follow the instructions in https://submit.mit.edu/submit-users-guide/program.html to set up a conda environment in your work directory for the software then select it as a kernel in the Jupyter notebook.

</details>

If running on subMIT, you will need to make the following change to **ALL** cells below where these appear

* change the line `source /opt/venv/bin/activate` to `source /work/submit/${USER}/venv816/bin/activate`
* change all references to the partition `pcpu` to `submit`

Now return to the beginning of this section to start the exercise in this cloud cluster session.

</details> -->

If running on an HPC cluster:  When you are all finished, make sure you close ALL windows you opened to reliquish the resoures you reserved on the cluster.  Close your browser window and `Terminal B`.  In `Terminal A`, hit the keyboard combo `Control-C` or `Command-C` repeatedly to shut down the Jupyter server, then type `exit` (followed by return) repeatedly until the terminal window closes (or hit the "X").

If running on an HPC cluster and you want to download your data locally (to your laptop), run the following cell to create a zip file of the data.  When the cell finishes, right-click on `trainingdata.zip` in the File Browser side panel (menu `View --> File Browser`) and select `Download`.

In [ ]:
import zipfile
import glob

files_to_zip = glob.glob('data_*.h5')
with zipfile.ZipFile('trainingdata.zip', 'w') as fout:
    for file in files_to_zip:
        fout.write(file)

`Summary of Key Concepts:`

* A serial calculation will *not* simply run faster on a multi-core machine; parallelization is required for speedup
* Develop the habit of looking for opportunities for parallelization in your *workflow*
* When implementing parallelization, always be mindful of contention
* Parallelization can shorten time to result and/or allow more sophisticated/expensive calculations to be performed

###### 1.2.1.1.2 Batch Jobs: Separate Jobs

In this section, we will submit and interact with jobs directly from this Jupyter notebook.  As such, it will be handy to launch this notebook on the cluster.  

Note:  If you are still running the notebook on the cluster from the previous section, please make sure to close and exit all sessions as instructed above before proceeding.  This is because your notebook was running in a multicore allocation in the previous section, and we will not use those resources here.  This is an important considerate best-practice: only request the resources you actually need & will use.  This will free those resource for others to use; clusters are shared resources.


Repeat the instructions in the previous section for "Running Jupyter on a CPU compute node" within "Instructions for running on cloud HPC Cluster", **except** in the `salloc` statement, use `-c 1` instead of the number specified in the instructions.  For your convenience, we repeat those instructions here.






<details>
<summary>Instructions for running on cloud HPC Cluster (click arrow to show/hide)</summary>

##### Initial Setup

If you have not yet completed the Initial Setup once, you must do that first.  Click the arrow below for instructions.  You should only have to do this *once* this semester (unless you get a new laptop).

<details>
<summary>Initial One-Time Setup Instructions (click arrow to show/hide)</summary>

1.  Install Google Cloud CLI: [text](https://cloud.google.com/sdk/docs/install)
  - Leave the default options for the various steps (install bundled python, run `gcloud init`, etc)
  - Eventually, a browser window will pop up asking you to sign into Google giving Google Cloud SDK access to your Google Account; do this.

2.  When you are finished with the browser window that pops up, close it *AND RETURN to the terminal window* which was open before the browser, and complete initialization there.  
  - *IF* there is no longer a terminal window open, run 
  open the Google Cloud CLI link which should have been installed on your desktop and then run `gcloud init` when that opens.  Choose the following:
  - Set your default `cloud project` to: `mit-teaching-project`
  - Set your default `Compute Region and Zone` to: `us-central1-b`

3.  It may be a good idea to restart your computer now; if you do not, keep in mind to try restarting if you run into any future problems.

</details>


##### Run Jupyter on a *CPU compute node*

We will launch Jupyter notebooks in a way which will give you 2 great tools:

* manually starting a Jupyter server (on a remote machine in a custom enviornment)

* ssh port forwarding (useful even beyond Jupyter notebooks)

<details>
<summary>More Detail (click arrow to show/hide)</summary>

Some clusters may have JupyterHUB set up.  This is a tool which can automatically perform these tasks for you, but it's still good to know how to do this yourself.  JupyterHUB usually provides predefined options; with the techniques we will show you, you can request custom resources.

</details>

Following these directions, your jupyter lab session (notebooks, terminal) will have access to all of the resources (cores, memory, ...) specified in your SLURM request (the `salloc` statement).  This setup will involve you opening/using 2 different terminal windows on your laptop; we will label them `Terminal A` and `Terminal B`.  

Note: if you are on a Windows machine, then by "terminal" below, we mean the window opened when you double-click on the `Google Cloud SDK Shell` icon on your Desktopm, or go to Start (Window button) and type in `Google Cloud SDK Shell`.  On a Mac, open the terminal by going to Launchpad, typing `Terminal` in Spotlight Search (CMD+Space), or in Finder open `Applications` then `Utilities` folders then click on Terminal.

In the terminal on your laptop (we will call this terminal window `Terminal A`), run the following comand (you may be prompted to enter your ssh key password)

```bash
gcloud compute ssh hpc-slurm-login-001 --tunnel-through-iap
```

<details>
<summary>If this is your first time connecting ... (click arrow to show/hide)</summary>

you may see a warning(s) like below:

```bash
WARNING: The private SSH key file for gcloud does not exist.
WARNING: The public SSH key file for gcloud does not exist.
WARNING: The PuTTY PPK SSH key file for gcloud does not exist.
WARNING: You do not have an SSH key for gcloud.
WARNING: SSH keygen will be executed to generate a key.
This tool needs to create the directory [C:\Users\muser2\.ssh] before being able to generate SSH keys.

Do you want to continue (Y/n)?
```

and/or perhaps like

```bash
The host key is not cached for this server:

compute.783492966651981141 (port 22)

You have no guarantee that the server is the computer you think it is.

The server's ssh-ed25519 key fingerprint is:

ssh-ed25519 255 SHA256:QHZjTMoUqO2yxfSn8UtkVcTpozMgvPQgfuTjmVAvQy0

If you trust this host, press "Accept" to add the key to PuTTY's cache and carry on connecting.
```

but with the numbers/characters different.  This is OK and expected.  Please continue.

</details>




<details>
<summary>If that did not work ... (click arrow to show/hide)</summary>

* If you have MIT Kerberos Ticket Manager installed, when you run the above, the connection may seem to hang (freeze).  This typically means a login window for MIT Kerberos Ticket Manager is open somewhere (often instigated by the above command).  Look to see if Kerberos Ticket Manager is open behind the terminal or other windows, or in another desktop; if so, hit cancel to close it.  Then the connection should proceede.

* If you already have Google Cloud CLI installed and use it for other projects, you will have to add these aruments to the above line to specify project name and region/zone this this course's project: `--project=mit-teaching-project --zone=us-central1-b`

</details>

This should connect you to a login node (you can confirm by typing `hostname` followed by return and it should print `hpc-slurm-login-001`).  Then run the following commands in that terminal (Note: Specific exercises instruct you to replace this `salloc` statement with a different one to request more resources)

```bash
salloc -N 1 -c 1
```

This requests `1` core on 1 node.  You may have to wait for a while after the `salloc` statement for a node to become free or a new virtual node to be created.  Once you are notified your resources are ready and get a terminal prompt again, run

```bash
hostname
```

Take note of the output printed by the `hostname` command; this is the name of the compute node on which your notebook will run.  Then run the following

```bash
source /opt/venv/bin/activate
jupyter lab --no-browser --port 8080
```

The first line will activate the python virtual environment in which we have installed the necessary software.  Virtual environments are a good practice for isolating/managing packages.  The second line will start your jupyter lab server and print a lot of text to the screen, including lines like `Or copy and paste one of these URLs:` followed by a url which looks like `http://localhost:8080/lab?token=[...]` (where the `...` is a long string of letters & numbers).  Take note of this URL.

Now open a new terminal window on your laptop (we will call this `Terminal B`) and run the following command in it

```bash
gcloud compute ssh hpc-computecpu-0 --tunnel-through-iap -- -L 8080:localhost:8080
```

where instead of `hpc-computecpu-0`, write the name of the compute node (the name returned by the `hostname` command in the earlier step above).  This should just log in to the cluster and look like a regular ssh session; when it does that, just leave it open.

Now paste the URL mentioned above into your web browser on your laptop.  This should open Jupyter Lab in your web browser.  Commands you run in this Jupyter session are actually being run *on the cluster node*.

If you want to run a particular notebook on this cluster, simply drag and drop the notebook file into the Files sidebar of the Jupyter window in your browser.  (If your notebook is on colab, you will have to download your notebook, then drag the downloaded file into Jupyter Lab).

<details>
<summary>Troubleshooting (click arrow to show/hide)</summary>
  
* If you get an error that the port is not available, then please start over again substituting a different number for `8080` above.

</details>

</details>




<!-- <details>
<summary>Instructions for running on Engaging HPC Cluster (click arrow to show/hide)</summary>

For more information and troubleshooting, see: https://orcd-docs.mit.edu 
##### Initial Setup

If you have not yet completed the Initial Setup once, you must do that first.  Click the arrow below for instructions.  You should only have to do this *once* this semester (unless you get a new laptop).

<details>
<summary>Initial One-Time Setup Instructions (click arrow to show/hide)</summary>

1. Follow instructions in https://orcd-docs.mit.edu to create an account.

1. Connect to the cluster by running in a terminal: `ssh <username>@orcd-login001.mit.edu` where you should replace `<username>` with your kerberos id (beginning part of your mit email address).

To open terminal: if you are on a Windows machine, go to Start (Window button) and type in `Terminal`.  On a Mac, open the terminal by going to Launchpad, typing `Terminal` in Spotlight Search (CMD+Space), or in Finder open `Applications` then `Utilities` folders then click on Terminal.

To open terminal: if you are on a Windows machine, go to Start (Window button) and type in `Terminal`.  On a Mac, open the terminal by going to Launchpad, typing `Terminal` in Spotlight Search (CMD+Space), or in Finder open `Applications` then `Utilities` folders then click on Terminal.

2. Make a private directory for this class by running the following commands

```bash
mkdir ~/private_816
chmod 700 ~/private_816
```

3. Install sofware by running

```bash

python3 -m venv ~/venv816
source ~/venv816/bin/activate
python3 -m pip install --upgrade pip
python3 -m pip install numpy scipy matplotlib tqdm imageio h5py ipykernel jupyterlab jupyter-server-proxy dask[distributed] pathos joblib ipyparallel ipywidgets torch torchvision torchaudio torchmetrics pyright python-language-server python-lsp-server

```
4.  You can log out now by running `exit` followed by Enter/Return

</details>


##### Run Jupyter on a *CPU compute node*

We will launch Jupyter notebooks in a way which will give you 2 great tools:

* manually starting a Jupyter server (on a remote machine in a custom enviornment)

* ssh port forwarding (useful even beyond Jupyter notebooks)

<details>
<summary>More Detail (click arrow to show/hide)</summary>

Some clusters may have JupyterHUB set up.  This is a tool which can automatically perform these tasks for you, but it's still good to know how to do this yourself.  JupyterHUB usually provides predefined options; with the techniques we will show you, you can request custom resources.

</details>

Following these directions, your jupyter lab session (notebooks, terminal) will have access to all of the resources (cores, memory, ...) specified in your SLURM request (the `salloc` statement).  This setup will involve you opening/using 2 different terminal windows on your laptop; we will label them `Terminal A` and `Terminal B`.

In the terminal on your laptop (we will call this terminal window `Terminal A`), run the following command but replace `<username>` with your kerberos id (beginning part of your mit email address):

```bash
ssh <username>@orcd-login001.mit.edu
```

This should connect you to a login node (you can confirm by typing `hostname` followed by return and it should print `orcd-login001`).  Then run the following command in that terminal

```bash
salloc -N 1 -p mit_quicktest -c 1
```

This requests `1` core on one node.  You may have to wait for a while after the `salloc` statement for the resources (cores on a node) to become available.  *Note:* This allocation will only last for 15 minutes!  If you want to work longer without needing to redo the steps to restablish the connection, you can replace `mit_quicktest` in the line above with `mit_normal`, but you may have to wait longer (or reduce the number of cores in the request, `-c <# cores>`).  Once a new prompt shows up, run

```bash
hostname
```

Take note of the output printed by the `hostname` command; this is the name of the compute node on which your notebook will run.  Then run the following

```bash
cd ~/private_816
source ~/venv816/bin/activate
jupyter lab --no-browser --ip=0.0.0.0  --port 8080
```

The first line will activate the python virtual environment in which you have installed the necessary software.  Virtual environments are a good practice for isolating/managing packages.  The second line will start your jupyter lab server and print a lot of text to the screen, including lines like `Or copy and paste one of these URLs:` followed by a url which looks like ` http://127.0.0.1:8080/lab?token=[...]` (where the `...` is a long string of letters & numbers).  Take note of this URL.

Now open a new terminal window on your laptop (we will call this `Terminal B`) and run the following command in it

```bash
ssh -L 8080:nodename:8080 <username>@orcd-login001.mit.edu
```

where instead of `nodename`, write the name of the compute node (the name returned by the `hostname` command in the earlier step above).

Now paste the URL mentioned above into your web browser on your laptop.  This should open Jupyter Lab in your web browser.  Commands you run in this Jupyter session are actually being run *on the cluster node*.

If you want to run a particular notebook on this cluster, simply drag and drop the notebook file into the Files sidebar of the Jupyter window in your browser.  (If your notebook is on colab, you will have to download your notebook, then drag the downloaded file into Jupyter Lab).

<details>
<summary>Troubleshooting (click arrow to show/hide)</summary>
  
* If you get an error that the port is not available, then please start over again substituting a different number for `8080` above.

</details>

*Note:* If running on Engaging, you will need to make the following change to **ALL** cells below where these appear

* change the line `source /opt/venv/bin/activate` to `source ~/venv816/bin/activate`
* change all references to the partition `pcpu` to `mit_quicktest` or `mit_normal`

</details>









<details>
<summary>Instructions for running on subMIT Cluster (click arrow to show/hide)</summary>

For more information and troubleshooting, see: https://submit.mit.edu 
##### Initial Setup

If you have not yet completed the Initial Setup once, you must do that first.  Click the arrow below for instructions.  You should only have to do this *once* this semester (unless you get a new laptop).

<details>
<summary>Initial One-Time Setup Instructions (click arrow to show/hide)</summary>

1. Follow instructions in https://submit.mit.edu/submit-users-guide/ to create an account.

1. Connect to the cluster by running in a terminal: `ssh <username>@submit.mit.edu` where you should replace `<username>` with your kerberos id (beginning part of your mit email address).

To open terminal: if you are on a Windows machine, go to Start (Window button) and type in `Terminal`.  On a Mac, open the terminal by going to Launchpad, typing `Terminal` in Spotlight Search (CMD+Space), or in Finder open `Applications` then `Utilities` folders then click on Terminal.

To open terminal: if you are on a Windows machine, go to Start (Window button) and type in `Terminal`.  On a Mac, open the terminal by going to Launchpad, typing `Terminal` in Spotlight Search (CMD+Space), or in Finder open `Applications` then `Utilities` folders then click on Terminal.

2. Make a private directory for this class by running the following commands

```bash
mkdir ~/private_816
chmod 700 ~/private_816
```

3. Install sofware by running

```bash

python3 -m venv /work/submit/${USER}/venv816
source /work/submit/${USER}/venv816/bin/activate
python3 -m pip install --upgrade pip
python3 -m pip install numpy scipy matplotlib tqdm imageio h5py ipykernel jupyterlab jupyter-server-proxy dask[distributed] pathos joblib ipyparallel ipywidgets torch torchvision torchaudio torchmetrics pyright python-language-server python-lsp-server

```
4.  You can log out now by running `exit` followed by Enter/Return

</details>


##### Run Jupyter on a *CPU compute node*

We will launch Jupyter notebooks in a way which will give you 2 great tools:

* manually starting a Jupyter server (on a remote machine in a custom enviornment)

* ssh port forwarding (useful even beyond Jupyter notebooks)

<details>
<summary>More Detail (click arrow to show/hide)</summary>

Some clusters may have JupyterHUB set up.  This is a tool which can automatically perform these tasks for you, but it's still good to know how to do this yourself.  JupyterHUB usually provides predefined options; with the techniques we will show you, you can request custom resources.

</details>

Following these directions, your jupyter lab session (notebooks, terminal) will have access to all of the resources (cores, memory, ...) specified in your SLURM request (the `salloc` statement).  This setup will involve you opening/using 2 different terminal windows on your laptop; we will label them `Terminal A` and `Terminal B`.

In the terminal on your laptop (we will call this terminal window `Terminal A`), run the following command but replace `<username>` with your kerberos id (beginning part of your mit email address):

```bash
ssh <username>@submit.mit.edu
```

This should connect you to a login node (you can confirm by typing `hostname` followed by return and it should print `orcd-login001`).  Then run the following command in that terminal

```bash
salloc -c 1
```

This requests `1` core on one node.  You may have to wait for a while after the `salloc` statement for the resources (cores on a node) to become available.  Once a new prompt shows up, run

```bash
hostname
```

Take note of the output printed by the `hostname` command; this is the name of the compute node on which your notebook will run.  Then run the following

```bash
cd ~/private_816
source /work/submit/${USER}/venv816/bin/activate
jupyter lab --no-browser --port 8080
```

The first line will activate the python virtual environment in which you have installed the necessary software.  Virtual environments are a good practice for isolating/managing packages.  The second line will start your jupyter lab server and print a lot of text to the screen, including lines like `Or copy and paste one of these URLs:` followed by a url which looks like ` http://127.0.0.1:8080/lab?token=[...]` (where the `...` is a long string of letters & numbers).  Take note of this URL.

Now open a new terminal window on your laptop (we will call this `Terminal B`) and run the following command in it

```bash
ssh -L 8080:localhost:8080 -J <username>@submit.mit.edu <username>@hostname.mit.edu
```

where instead of `nodename`, write the name of the compute node (the name returned by the `hostname` command in the earlier step above).

Now paste the URL mentioned above into your web browser on your laptop.  This should open Jupyter Lab in your web browser.  Commands you run in this Jupyter session are actually being run *on the cluster node*.

If you want to run a particular notebook on this cluster, simply drag and drop the notebook file into the Files sidebar of the Jupyter window in your browser.  (If your notebook is on colab, you will have to download your notebook, then drag the downloaded file into Jupyter Lab).

<details>
<summary>Troubleshooting (click arrow to show/hide)</summary>
  
* If you get an error that the port is not available, then please start over again substituting a different number for `8080` above.

* If this still does not work, you can get a mulit-cpu Jupyter session via JupyterHub at http://submit.mit.edu/jupyter  In this case, you will have to follow the instructions in https://submit.mit.edu/submit-users-guide/program.html to set up a conda environment in your work directory for the software then select it as a kernel in the Jupyter notebook.

</details>

If running on subMIT, you will need to make the following change to **ALL** cells below where these appear

* change the line `source /opt/venv/bin/activate` to `source /work/submit/${USER}/venv816/bin/activate`
* change all references to the partition `pcpu` to `submit`

</details> -->


Now that this notebook is running on the cluster, before we begin, let's place this section in its own directory

In [ ]:
!mkdir -p ~/batch_separate_T && chmod 700 ~/batch_separate_T
%cd ~/batch_separate_T

As described in `More Detail: HPC Clusters` at the end of the previous section, a "batch job" is a calculation that is run without human interaction using certain dedicated resources (typically specified in a batch submission script) and is submitted to a resource-manager/scheduler where it will often wait in a queue until it runs at a later time.

As such, we must first move the relevant code from this notebook into a script file, which can be executed from the command line and will run without human interaction.  We want the script to calculate *one* instance of the parallel unit; in this case, we want to create a script which will simulate one given temperature.  Specifically, we will create a file `simT.py` which we can run in the terminal as `python3 simT <temp>` where `<temp>` is an integer defining the temperature index as above.

<details>
<summary>More Detail (click arrow to show/hide)</summary>

Here we chose to pass the desired inputs to our script via command-line arguments, but we could have done it differently.  Environment variables and input files are other common methods.

</details>

Basically, we want our script to execute the function `simulate_single_T` defined above, but since the cell defining this function depends on previously-run cells, this task is not as simple as just copying and pasting the cell's code into a text file.  Instead, we need to gather all the necessary pieces of code from this notebook.  We have outlined the necessary pieces for you in the cell below.  This cell begins with a "magic" line, `%%writefile simT.py`, which prints the contents of the cell into the script file `simT.py` when the cell is run.

Complete this cell by copying & pasting your code from above, guided by the comments:

<!-- `Hint:` Comment-out the top `%%writefile simall.py` when collecting your code.  This will allow your IDE to treat the code as code (highlighting, navigation, ...) rather than text.  When you are finished, uncomment the top line again. -->

In [ ]:
%%writefile simT.py
# Package up self-sufficient code
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
import h5py
import os
import sys # for getting command-line arguments


# Copy your hamiltonian() function here

# Copy your flip() function here

# Copy your mag() function here

# Copy your update_rand() function here

# Copy your Ising class here


# We copied this for you from above. Do not change
def simulate_single_T(temp, T):
    ''' temp = index to run in array T of temperatures'''
    #Some hacky code to clean up things
    filename='data_'+str(T[temp])+'.h5'
    try:
        os.remove(filename)
    except OSError:
        pass
    test = Ising(32, T[temp])
    test.nsim=500 #10#500 =>
    test.simulate_save()


# New piece of code to run via command line with temp as an argument
# i.e. you can run this in the terminal as:
# python simT.py 1
if __name__ == "__main__":
    nt=20
    T = np.round(np.linspace(1.53, 3.28, nt),2)
    # Get the command-line argument (index of T)
    temp = int(sys.argv[1])
    simulate_single_T(temp, T)

It's easier to debug interactively in a Jupyter notebook, so let's validate the script now following these steps:
1. Set `test.nsim` to a small value in the above cell
1. Select the menu item "Runtime --> Disconnect and delete runtime"
1. Run the cell above.  This will write the script file
1. Run the cell below to make sure it runs a single temperature simulation
1. Once that works, set `test.nsim` back to a large value & **re-run** the above cell

Doing this will force the following cell to only consider what is in the file `simT.py`, not what we previously ran in the notebook.  Note the commands in this next cell are not python commands, so we add the `%%bash` magic line to the top to run the cell's lines as if you were in a terminal.

<details>
<summary>More Detail (click arrow to show/hide)</summary>

It's handy to learn how to work in the terminal, so we use terminal commands with `%%bash` magic.  A few alternatives to using the `%%bash` magic include:

* Place a `!` at the beginning of each line in a regular cell
* Make a Python script which uses `os` features such as `subprocess.run` to run these commands
* In Jupyter lab, from the top menu select `File --> New --> Terminal`, then enter the commands from any `%%bash` cell, excluding the actual `%%bash` line itself

</details>


In [ ]:
%%bash
rm -rf tmp_test    # remove test directory if exists
mkdir tmp_test     # test in temporary directory
cd tmp_test        # so you don't overwrite your work
cp ../simT.py .    # put your script in the test directory
python3 simT.py 1  # run the script to test it

Now let's make job scripts.  For our purposes, a job is an independent set of computational taks.  A job script defines the requirements for the job (`#SBATCH` lines request # CPUs, GPUs, memory, time, ...) and the bottom lists the actual commands to be run during the job execution.  Each job will be added to the SLURM scheduler's *queue* and will execute at some later time; each job is indepenent so jobs may run simultaneously, or sequentially in *any* order.  In general, clusters have multiple *partitions*, or distinct groups of machines.  Different partitions may exist for short debugging tests, long CPU calculations, GPU calculations, or whole-node vs few-core jobs.  Let's define a function which will write one job script (for a particular temp)


<details>
<summary>More Detail (click arrow to show/hide)</summary>

We use `textwrap.dedent` so that we can define a multiline string lined up with the function definition but remove the indentation in our final string.  Alternatives include concatenating the string line by line, or not indenting the string.

The SLURM scheduler plays the complicated Tetris game of scheduling resources.  Each system's scheduler may be configured differently; for instance one system may give higher priority to certain groups, and others may give priority based on recent usage (more usage means lower priority).  Some systems are configured to only run jobs that use an entire node, whereas some, like ours, allow single-core jobs.  When moving to a new cluster it's always important to read the system's documentation.

We use a very simple job script for a simple task.  SLURM job scripts support more advanced features such as dependencies, which can enforce a sequential nature to jobs, requiring that certain jobs be complete before others begin.  Job scripts may also allocate multiple resources and run certain commands on a subset.  There are many more features; you can read about this in the SLURM documentation.

</details>


In [ ]:
import textwrap
def write_job_T(temp, partition='pcpu'):
    '''Creates a job script to run a single temperature'''
    job_script = f"""\
    #!/bin/bash
    #SBATCH --job-name=ising_{temp}
    #SBATCH --output=ising_{temp}_out_%j.txt
    #SBATCH --error=ising_{temp}_err_%j.txt
    #SBATCH --partition={partition}
    #SBATCH --nodes=1
    #SBATCH --ntasks-per-node=1
    #SBATCH --cpus-per-task=1
    #SBATCH --time=00:15:00
    #SBATCH --mem-per-cpu=1G

    # Exeution portion of script below here
    cd $SLURM_SUBMIT_DIR  # go to the directory where the job was submitted

    # IMPORTANT: Load your software environment
    # Typically includes `source`, `.`, `module load`, `conda activate` (or `conda run`) commands
    source /opt/venv/bin/activate

    # Let's print some info just for instructional purposes
    echo "Hi.  I am job: $SLURM_JOB_NAME with job id: $SLURM_JOB_ID running on node: $(hostname)"
    echo "I have access to $SLURM_CPUS_ON_NODE CPU(s) on this node."

    # Now the actual terminal command to run: simulate single T
    python3 simT.py {temp}
    """
    # Remove the indent from our multiline string above
    job_script = textwrap.dedent(job_script)  # Remove leading whitespace
    
    # Write the job script to a file
    with open(f"ising_{temp}.sh", "w") as f:
        f.write(job_script)

<details>
<summary>Some of the job script looks unfamiliar (click arrow to show/hide)</summary>

The job script is a shell script (BASH script here), not a python script.  Lines here represent Linux commands you would type into a terminal.  The `echo` statement functions here like a `print` statement in Python.  Values of variables are accessed (aka expanded/substituted) by placing a `$` in front of their name.  `SLURM_JOB_NAME`, `SLURM_CPUS_ON_NODE`, etc are known as environment variables and get assigned values when the script runs.  Placing `$VARNAME` inside a string in an echo statement is like f-strings in Python; sometimes they are written with {} and look even more like f-strings: `echo "I am job ${SLURM_JOB_NAME}."`.  One of our strings also contains a `" ... $(command) ..."` which will, in short, execute the command and place the output in the string; in our example, `hostname` is the command.

The output file, `#SBATCH --output=...`, will contain the standard output which was previously displayed on the screen in interactive mode.  Likewise, the error file, `#SBATCH --error=...`, will contain the error messages which were previously displayed on the screen in interactive mode.  The file specification each contain `%j`, which will be replaced by the job id when the job runs.  This ensures each file is unique, even if the same job is run multiple times, e.g. when troubleshooting.

</details>

Now we will write a separate job script for each temperature.  (In the next section we will show you a better way for this workflow, but it's instructive to do this first).

In [ ]:
nt = 20
for temp in range(nt):
    write_job_T(temp, partition='pcpu')

!ls ising_*.sh
!echo " " ; echo "example ising_10.sh contents:"
!cat ising_10.sh

Before you run "at scale", always do small tests! Submit a *single* job as a test.  You can run the following `sbatch` command in a terminal (make sure you are in this directory), or execute the following cell, which should return something like `Submitted batch job 12345` but instead of `12345`, it will be a different integer.  This integer is the job id; take note of it.

In [ ]:
%%bash
sbatch ising_10.sh

<details>
<summary>Troubleshooting ... (click arrow to show/hide)</summary>

If your request categorically cannot be satisfied, you will recieve an error when running `sbatch`.  Perhaps the most common for this example would be if your cluster has a time limit on the default partition which is shorter than what we requested in the job script.  I that case, you can decrease the `--time` portion of the job script, or run the job in a different partition using the `-p` option in `sbatch` or changing the partition used in the script iself.

</details>

To check on the status of your jobs, use the `squeue` command (in terminal or cell)

In [ ]:
%%bash 
squeue

Each job will appear as a separate row.  The ST (state) column indicates whether a job is pending (PD), running (R), configuring (CF), or other.  Recall, you will see an "interactive" job in which this Jupyter notebook is running.  To see only your jobs, use `squeue --me` or `squeue -u $USER`

Run the following command, replacing `12345` with the actual job id from your `sbatch` call to get detailed information on your job.  The `JobState` portion will tell whether the job has run yet or not.

In [ ]:
%%bash
scontrol show job 12345


Other useful SLURM commands, assuming job id 12345, include:

* `scancel 12345` = cancel the job
* `scontrol suspend 12345` = suspend (pause/hold) the job
* `sstat -j 12345` = statistics of a running job
* `sacct` = information about your past / completed jobs (some fields only accurate after a job finhes)
* `seff 12345` = efficiency report for a job (only reliable after job is complete)
* `sinfo` = print information about the *cluster* (e.g. partitions)

All these are shell commands; they are commonly run directly in the terminal after ssh-ing into the cluster, but you can also run them from within a Jupyter notebook (as we do here) using the `%%bash` magic or `!` in a Jupyter cell.  See SLURM documentation for more info (or manual page, e.g. run `man sinfo` in terminal).

<details>
<summary>Some Examples (click arrow to show/hide)</summary>

* `sinfo -Ne -O "PARTITION:.10,NodeHost:.20,StateLong:.11,NodeAIOT:.15,CPUsState:.15,Memory:.9,AllocMem:.9"` to see machine details in cluster
* for a running job: `sstat -j 12345.batch --format=JobID,MaxRSS,AveRSS,MaxVMSize,AveVMSize` 
* for a completed job: `sacct -j 12345 --format=JobID,JobName,Elapsed,TotalCPU,UserCPU,SystemCPU,MaxRSS,ReqMem,State`
* `sacct --user=$USER --starttime=now-1day --format="User,NodeList,Start,Elapsed,State,Reason"` to look at your jobs in the past day

</details>

Once your job has started, you can view the job output & error logs, `ising_10.out` & `ising_10.err` respectively: navigate in the File Browser sidebar in Jupyter (`View --> File Browser` if not visible) to the `batch_separate_T` directory and double-click to open each file.

<details>
<summary>Troubleshooting Tips (click arrow to show/hide)</summary>

To troubleshoot, use the above SLURM commands to see if the job is stuck in the queue; the `Reason` field in `scontrol show job` or `sacct` can be helpful.  Also, look at the contents of the error file as outlined above.

Note: the compute nodes on the cloud HPC cluster are *preemptible* (aka spot VMs) meaning that, at any given time, they may be taken offline suddenly if a higher priority user requests them.  This is a possible reason for job failure.

If your job uses more memory or time than requested, it will be terminated by the scheduler.  Increase your time request by modifying the `#SBATCH --time` line in the job script.

</details>

After the `ising_10` job finishes and looks OK, run the next cell to run the full set of jobs.  Recall: you do not want duplicate `ising_10` jobs running simultaneously as it may lead to contention problems for the output file.

In [ ]:
import subprocess, time
for temp in range(nt):
    subprocess.run(["sbatch", f"ising_{temp}.sh"])  # sbatch <script file>
    time.sleep(0.5)  # sleep for half a second to avoid overwhelming the scheduler

Now check on your jobs in the queue

In [ ]:
!squeue --me

Note: In the cloud cluster, jobs may spend a while in the configuring CF state while new nodes are being created.  The cloud cluster is "elastic" so new compute nodes are dynamically created as needed; this may take a while.

Once all your jobs complete, you should now have a complete set of output files!

<details>
<summary>Important: Before you apply this to a different problem ... (click arrow to show/hide)</summary>

TL;DR - Always consult the system administrator (help desk) of your cluster to see what is the expected/optimal number, size, and shape of jobs for that particular cluster

Above, we had a modest number of temperatures to run, 20.  If we had many temperatures to run, say 2000, and each temperature is a relatively short calculation, we may not want to handle them each as a separate SLURM job.  First, there is overhead in the scheduler considering and setting up each job, plus overhead in each job starting up the software for that job, e.g. loading the necessary python modules.  It would be far more efficient both for the scheduler and your code to run a for-loop over temperatures within the python script.  This would result in fewer jobs for the scheduler to have to consider & juggle, and the overhead of loading python modules would only need to be performed once per job instead of once per temperature.  For instance, each job could loop over 1000 temperatures.  The number of jobs can be further reduced by using multiprocessing within each job (see examples above and below).  In that case, each python process could loop over many temperatures, and many processes could be running within a single SLURM job.

Also, in this case, since all the jobs are similar, a better way to submit these jobs would be via a job array; see next section.

Contact the system administrator (help desk) of your cluster to see what is the expected/optimal number, size, and shape of jobs for that particular cluster.  Some clusters cater to whole-node jobs (multiprocessing could be handy) while some cater to many single-core jobs.

Side note: There are resources, often known as High-Throughput resources, which are often configured for many small separate single-core jobs.  These are often handled via a different resource manager, HTCondor, or SLURM configured specially for so-called High-Througput workflows.

</details>

###### 1.2.1.1.3 Batch Jobs: Job Arrays

In general, each job script can contain an arbitrary set of commands, completely unrelated to those in other scripts.  In the previous section, however, the job scripts were very similar, only differing in input command line arguments.  This particular kind of workflow *can* be done as separate jobs as above, but SLURM provides a special tool for this kind of workflow: Job Arrays.  In general, if you find yourself running for-loops to create job scripts, you should consider using SLURM Job Arrays instead.

Assuming you have run the previous section (this notebook is running on the cluster), let's set this up in its own directory and copy the relevant file, `simT.py`

In [ ]:
!mkdir -p ~/batch_job_array && chmod 700 ~/batch_job_array
%cd ~/batch_job_array
!cp ~/batch_separate_T/simT.py ./  # copy your script to this new directory

Now run the next cell to create the (single!) job array script file

In [ ]:
%%writefile ising_job_array.sh
#!/bin/bash
#SBATCH --job-name=ising_array
#SBATCH --output=ising_%a_out_%A.txt
#SBATCH --error=ising_%a_err_%A.txt
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=1
#SBATCH --time=00:15:00
#SBATCH --mem-per-cpu=1G
#SBATCH --array=0-19  # Array job for all temperatures

# Exeution portion of script below here
cd $SLURM_SUBMIT_DIR  # go to the directory where the job was submitted

# IMPORTANT: Load your software environment
# Typically includes `source`, `.`, `module load`, `conda activate` (or `conda run`) commands
source /opt/venv/bin/activate

# Let's print some info just for instructional purposes
echo "I am running on $(hostname) and am part of SLURM job array: $SLURM_ARRAY_JOB_ID "
echo "Specifically, I am task # $SLURM_ARRAY_TASK_ID out of a total of $SLURM_ARRAY_TASK_COUNT tasks in this job array."

# Now the actual terminal command to run: simulate single T
python3 simT.py $SLURM_ARRAY_TASK_ID

<details>
<summary>More Details (click arrow to show/hide)</summary>

Note: If you want to run on a non-default partition, you'll need to add an `#SBATCH --partition` line as in the previous example.

The `--array` statement (aka `-a`) can support 

* lists of distinct values: `--array=0,2,3,7`
* combinations: `--array=0,3,10-15`
* limit the maximum number of simultaneously-running tasks: `--array=0-19%4` limits to `4` simultaneously-running tasks

In the separate-job example in the previous section, each output file `SBATCH` specification ended in `%j.txt` since the job id, `%j`, was a unique identifier for each job.  For job arrays, the unique identifer is now the *combination* of `%A` and `%a`, the ArrayJobID and ArrayTaskID, respectively.

See more info at: https://slurm.schedmd.com/job_array.html 

</details>

Run the next cell to submit this job array

In [ ]:
!sbatch ising_job_array.sh

Wait a few seconds, then run the next cell to check on your jobs.  You can run the cell again later to check again.  Also check out the output/error logs for the tasks as described for the separate jobs.

In [ ]:
!squeue --me

Note that SLURM creates all 20 tasks, and when each executes, it has a different value assigned to the environment variable `SLURM_ARRAY_TASK_ID`.  Thus, `SLURM_ARRAY_TASK_ID` plays the role of the variable `temp` in earlier sections, and SLURM's automatic handling of job arrays takes the place of our explicit for-loop in the previous section.

One advantage of using job arrays is that one can easily select all or a subset of a job array.  For instance, let's say your ArrayJobID is `145` and you only want to cancel ArrayTaskID's `7` through `9`, you could do this with `scancel 145_[7-9]`.  Arrays can also be helpful when using advanced features such as job dependencies.

###### 1.2.1.1.4 Batch Jobs: Multiprocessing

Last but not least, we can of course run our earlier multiprocessing calculation in batch mode.  In this case, we will submit a single job which requests & uses several cores.  This time, however, we must still use our `simT.py` script file, because the batch job will have no knowledge of this notebook and what was previously run in it.

<!-- <details>
<summary>Aside ... (click arrow to show/hide)</summary>

Now that we have pulled out our function definition into a separate file, `simT.py`, we could actually use the built-in Python multiprocessing tools in a Jupyter notebook.  The definition of the function in a separate file circumvents the pickling issues which prevented our straightforward use of the module earlier.

</details> -->

First, assuming you have run the separate-batch-job section above (this notebook is running on the cluster), let's set up a separate directory for this section and copy `simT.py` here

In [ ]:
!mkdir -p ~/batch_multiprocessing && chmod 700 ~/batch_multiprocessing
%cd ~/batch_multiprocessing
!cp ~/batch_separate_T/simT.py ./  # copy your script to this new directory

First we must add the python multiprocessing code to a script file which can be run in batch mode.  Since we already have `simT.py` from above, let's add this multiprocessing part to a new, separate python file, `simMulti.py`.  This is just a choice; we *could* put everything in one file.  

We copy & paste from the cell in the above interactive multiprocessing section, but placing the contents within a `if __name__ == "__main__":` block (important).  Also, instead of copying the `simulate_single_T` from above, we add an import statement to pull relevant code in from `simT.py`, which also contains all the supporting code.

In [ ]:
%%writefile simMulti.py
import os, time, logging
from dask.distributed import Client, LocalCluster
import numpy as np
from simT import simulate_single_T  # NEW: pull in code from simT.py
logging.getLogger("distributed").setLevel(logging.WARNING)

if __name__ == "__main__":
    # Start with the same code from the interactive multiprocessing section above
    nt=20
    T = np.round(np.linspace(1.53, 3.28, nt),2)
    print(T)

    # Instead of defining simulate_single_T here, we imported it from simT.py
    # This is because simT.py contains all the supporting code that function requires

    # Now replace the loop w/ new code to run in parallel ...
    list_temp = range(nt)  # Make list of inputs to define what each process (worker) will run
    list_T = [T]*nt        # Make list of other input: just repeat this one (same for each worker)
    # Run the calculation in parallel using Dask (see earlier example)
    cluster = LocalCluster() ; client = Client(cluster)
    futures = client.map(simulate_single_T, list_temp, list_T)
    client.gather(futures)
    client.close() ; cluster.close()

Next run the following cell to create a single job file.  Note this `#SBATCH --cpus-per-task` line now requests `8` cores on a single node.  

In [ ]:
%%writefile ising_multi.sh
#!/bin/bash
#SBATCH --job-name=ising_multi
#SBATCH --output=ising_multi_out_%j.txt
#SBATCH --error=ising_multi_err_%j.txt
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=8
#SBATCH --time=00:15:00
#SBATCH --mem-per-cpu=1G

# Exeution portion of script below here
cd $SLURM_SUBMIT_DIR  # go to the directory where the job was submitted

# IMPORTANT: Load your software environment
# Typically includes `source`, `.`, `module load`, `conda activate` (or `conda run`) commands
source /opt/venv/bin/activate

# Let's print some info just for instructional purposes
echo "Hi.  I am job: $SLURM_JOB_NAME with job id: $SLURM_JOB_ID running on node: $(hostname)"
echo "I have access to $SLURM_CPUS_ON_NODE CPU(s) on this node."

# Now the actual terminal command to run: simulate single T
python3 simMulti.py

Let's just try running this as a batch job.  If there are any problems, you can try running this interactively to debug, as we did above in the earlier sections.

In [ ]:
!sbatch ising_multi.sh

As usual, wait a few seconds, then run the next cell to check on your jobs.  You can run the cell again later to check again.  Also check out the output/error logs for the tasks as described for the separate jobs.  


<details>
<summary>Waiting a long time for the job to start? (click arrow to show/hide)</summary>

This job *may* sit a bit longer in the queue since it requires all the requested cores to be available at the same time on one node; this is a more restrictive request than our jobs in the previous sections.  You can try requesting fewer cores, the `--cpus-per-task` line.  If running on a local cluster, try to make sure you are running in a debugging/testing partition or QoS, if possible (ask your system admin for help).

</details>

In [ ]:
!squeue --me

Once this completes successfully, it's always a good idea to run `seff <jobid>` to make sure you actually used all the CPUs you requested; total run time is a good indicator of this as well. You have now run this calculation in parallel 4 different ways!  See also the HPC section in the Normalizing Flows section below for an example using GPUs.

How do you make the choice between running this calculation using multiprocessing in a single batch job as opposed to separate batch jobs for each temperature?  This choice is often determined by typical useage of the cluster on which you will run.  Some clusters cater to, or even require, whole-node jobs, in which case multiprocessing would be more natural.  Recall, there are many tools for multiprocessing and separate tasks can even be launched using slurm commands (not covered here).  Other clusters are heavily used by single-core jobs; if that is the case, a multi-core job may sit longer in the queue, waiting for all cores to be available simultaneously.  This is particularly true if the single-core jobs are long-running; other single-core jobs may slip in front of you since they fit the open slots easier as the scheduler plays Tetris.  Some clusters limit the number of jobs you may submit.  When in doubt, consult your cluster's documentation, and/or reach out to your system admin.

Interwoven above, there were comments on things to consider before applying these techniques to other problems; please read those before applying these techniques to other problems.

Before scaling up your workflow to an HPC environment, the first step is usually to optimize your code.  The time you spend on this depends on how many total cpu-hours you intend to run.  Just a quick or on-eoff test may be fine to parallelize without optimizing.  But if you plan to run many many cpu-hours, please be considerate to other users of your shared cluster and to the environment by optimizing your code first.  One step is to use fast third-party tools, many of which run compiled code under the hood, rather than running a code which has many loops in native python code.  There are olso options to compile or just-in-time compile your Python code.

<!-- When running interactively, multiprocessing can often be handier in that all of your cores are reserved for your entire session; no need to worry about being delayed by someone submitting many jobs as you are working. -->

If you have not done so already, you'll want to download this data to your local machine (e.g. laptop) so you can use it to complete this notebook.  To download, run the following cell to create a zip file of the data.  When the cell finishes, right-click on `trainingdata.zip` in the File Browser side panel (menu `View --> File Browser`) and select `Download`.

In [ ]:
import zipfile
import glob

files_to_zip = glob.glob('data_*.h5')
with zipfile.ZipFile('trainingdata.zip', 'w') as fout:
    for file in files_to_zip:
        fout.write(file)

Please make sure you close ALL windows you opened to reliquish the resoures you reserved on the cluster.  Close your browser window and `Terminal B`.  In `Terminal A`, hit the keyboard combo `Control-C` or `Command-C` repeatedly to shut down the Jupyter server, then type `exit` (followed by return) repeatedly until the terminal window closes (or hit the "X").  On your latptop, you can unzip `trainingdata.zip` to complete this notebook on your laptop (or where you usually work).

##### 1.2.1.2 Perform the training

This resumes the required portion of the exercise.

Now we would like to develop a neural network that will take in this dataaset and train for whether the sample has undergood a phase transition.

Before we build the Neural Network, we are going to use Torch Dataset to process and format the data. Setting this up is a little annoying so we will just write the code for this down here, and we will provide a little example of how to read a Torch DataLoaer.

The one part that we will leave for you to figure out is what are the right labels and how do you split up the testing and training dataset to ensure your network learns.

In [ ]:
class DataSet(Dataset):
    def __init__(self, samples, labels, temps):
        super(DataSet, self).__init__()
        self.labels  = labels
        self.samples = samples
        self.temps   = temps
        if len(samples) != len(labels):
            raise ValueError(
                f"should have the same number of samples({len(samples)}) as there are labels({len(labels)})")
            
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        y = self.labels[index]
        x = self.samples[index]
        t = self.temps[index]
        return x, y, t


#Here is some code to read all the different files and make a dataset
all_data  = None
all_temps = None
for temp in tqdm (range (nt), desc="Loading..."):
    f = h5py.File('data_'+str(T[temp])+'.h5', 'r')
    if temp == 0:
        all_data  = f['data']
        all_temps = np.ones(all_data.shape[0])*temp
    else:
        all_data  = np.append(all_data, f['data'],axis=0)
        all_temps = np.append(all_temps,np.ones(f['data'].shape[0])*temp)
    
all_data    = np.reshape(all_data,(all_data.shape[0],all_data.shape[1]*all_data.shape[2]))
all_labels  = #build a numpy array that has labels 1 for below phase transition and 0 for above transition
all_dataset = DataSet(samples=all_data.astype("float32"),labels=all_labels,temps=all_temps)

#Finally, we will split the dataset randomly
data_train, data_test = #code to randomly split test and training
#And a loader
batch=10
train_loader = DataLoader(data_train, batch_size=batch,shuffle=True)
#here is an example how it works
for count, (x, y, t) in enumerate(train_loader):
    print(count,"x value:",x,x.shape,"\n Label:",y,y.shape,"\n Temp:",t,t.shape)
    if count > 2:
        break

Now finally, setup a neural network that reads in the inputs, and then trains to find the label. Create your own neural network that takes the input as a vector or image (note you can use a Dense network or a CNN).

Additionally, add a loss that classifies whether the data is above or below the critical tempeature. Run the training so that we get good performance for the classification of above or below the critical temperature.

In [ ]:
class simple_MLP_4layer(torch.nn.Module):
    def __init__(self,input_size,out_channels=1,nhidden=64):
        super().__init__()
        self.model = nn.Sequential(
            #Design your own neural network
        )
        self.output  = ##Perhaps an activation between 0 and 1

    def forward(self, x):
        x = self.model(x)
        x = self.output(x)
        return x

                
def train(model,n_epochs=20):
    opt       = torch.optim.Adam(model.parameters(),lr=0.005)
    criterion = #### come up with a loss function that is appropriate
    for epoch in range(n_epochs):
        model.train(True)
        running_loss = 0.0; updates=0
        for x, y, t in train_loader:
            opt.zero_grad()
            y_hat = model(x)
            loss  = criterion(y_hat.flatten(),y) 
            loss.backward()
            opt.step()
            running_loss += loss
            updates +=1
            del x,y
        print('Epoch: {} LOSS train: {} '.format(epoch,running_loss/(updates*batch)))

model     = simple_MLP_4layer(all_data.shape[1],out_channels=1,act_out=True) 
train(model,n_epochs=400)

Now run the validation and confirm that temperature prediction of the neural network. Plot the true value of the vadliation set agains the prediction. How clos is the preidction? What is the accuracy

In [ ]:
model.train(False)
test_accuracy = Accuracy(task="binary", num_classes=2)
for x, y, t in test_loader:
    with torch.no_grad():
            #Run inference on test dataset    
            #Insert code here

#compute accuracy
#plot Prediction vs tempature for truth and pred
#Do you get good validation? 

#### 1.2.2 NN application on Triangle Ising Model

Ok, so once our validation is looking good, we want to show are neural network has actually learned something non-trivial for that we want to make a different Magnetic fild configuration with a different, but similar Hamiltonian, and see if we can us our neural network to predict the temuprature change.

As a consequence, write a function for the Hamiltonian of the Triangular Ising model. We can define the triangular Ising model as a trigualr array as oppose to a square array like in the picture below fore [here](https://link.springer.com/article/10.1134/S1063776122050016)
![triangular_lattice.webp](attachment:triangular_lattice.webp)

The Hamiltonian is again the Ising Hamiltonian except now we sum over all 6 elements $i,j$ connected to a point.
$$
H = -\frac{1}{2}\sum_{i,j\in{adjacent to i}} \sigma_{i}\sigma_{j} - h \sum_{i} \sigma_{i}
$$

Just fousing on teh first term, write a Hamiltonian that will act on an N$\times$N array following a pattern above  to create the triangular array. We can again use a video game Geometry to make this easier. Connect one end to the other.


In [ ]:
#Now lets define the trignular ising 
def hamiltonian(iArr,N):
    #Compute the hamiltonian (note if you use this function you don't need to redeclare the above class)
    energy = 0
    for i in range(N):
        for j in range(N):
     
    return energy

def flip(i,j,iArr,beta):
    #finally run the Markov Chain process that computes delta energy, and based on the Markov deicison flips the spins. 

#Run some checks? 
#test.simulate()
#E[temp],M[temp],C[temp],X[temp]=test.lastAvg()

Finally, run the neural entwork on a dataset for triangular Ising models. Show that you can predict the correct tempearture, what is it?

In [ ]:
T       = np.round(np.linspace(1.53, 6.86, 28),2);
for temp in tqdm (range (9), desc="Loading..."):
    filename='tridata_'+str(T[temp])+'.h5'
    try:
        os.remove(filename)
    except OSError:
        pass
    test = Ising(32,T[temp])
    test.nsim=500
    test.simulate_save('tri')

Finally, run the neural network on a dataset for triangular Ising models. Show that you can predict the correct tempearture, what is it?

In [ ]:
#Now lets test it on something different
def load():
    #following above, read teh data and output a dataset
    return data_tri_train, data_tri_test

data_tri_train, data_tri_test = load()
batch=1000
test_tri_loader = DataLoader(data_tri_test, batch_size=batch,shuffle=True)

#again following above, plot the temp applying your prevoiusly trained NN to the triagle boltzman
model.train(False)
for x, y, t in test_tri_loader:
    with torch.no_grad():

#What is accuracy?     
#plot Score vs Temp where is the phase transition, is it consistent with truth see arxiv paper

Ok, so we should now have a result, and you can see hwo this behaves. From this we can ask a few questions:  
 * Do you see a phase transition?  
 * Does this agree with the paper?
 * What can you do to make this more accurate?

Now there are many follow ups to this study. However, I would like to highlight that he big gains that come from thsi are not the network itself, its more that the NN has been able to do a visual inspection of a material and made conclusions that are not necessarily obvious. Using this can lead to better analysis of data. Moreover, we can ask ourselves the question if we can accurately predict properties of materials can we use this to advance our understanding.

In the following section, we will explore how we can use Machine learned distributions to rapdily accelerate simulation allow us to perform advanced lattice measurements leading to critical material properties.

# Part 2: Scalar field theory on a lattice

In this part of the tutorial, we will explore the methods of lattice Quantum Chromodynamics. Quantum Chromodynamics is the theory that describes the strong interaction between quarks and gluons, which are fundamental particles that make up mesons and hadrons. These particles constitute the majority of visible matter in the Universe. QCD is a crucial component of the Standard Model (SM) and is used to investigate physical phenomena such as the anomalous magnetic moment of the muon and the interactions of Dark Matter with Nuclei.

Due to its large coupling constant, QCD cannot be studied using perturbation theory. Well-established non-perturbative method, lattice QCD enables computations of physical observables though numerical simulations. Some of the most famous results obtained with lattice QCD are the study of quark potential, hadron spectrum, and hadron structure.

In this part of the tutorial, we will focus on the toy lattice QCD model - the lattice scalar field theory. Despite being the simplest interacting theory, it is also one of the most effective theoretical tools for studying critical phenomena in a wide variety of physical systems. Additionally, the Higgs field in the SM is a scalar field that breaks the weak isospin symmetry of the electroweak interaction and generates mass to many particles.

Instead of using the traditional lattice QCD approach, we will explore recent developments in machine learning. This method has already been applied to some toy models and has demonstrated its effectiveness by significantly reducing the uncertainties of physical observables by several orders of magnitude while guaranteeing correctness [arXiv:2003.06413, arXiv:2202.11712].

### Similarities between Scalar field theory and Ising Model

The Ising model and scalar field theory are seemingly different models at the microscopic level, but they share some fundamental properties that make them belong to the same universality class.

One of the key similarities is the global symmetry $Z_2$, which is the symmetry of the Ising model with respect to the flipping of all spins. In scalar field theory, this symmetry corresponds to the invariance of the Lagrangian under the transformation $\phi \rightarrow -\phi$, where $\phi$ is the scalar field.

Another important property shared by the two models is that they both undergo a phase transition at a critical temperature or coupling strength, respectively. At this critical point, the correlation length diverges, and the system exhibits scaling behavior that is independent of the microscopic details of the system.

This scaling behavior is described by critical exponents, which characterize the power-law behavior of various physical observables near the critical point. These critical exponents are universal, meaning that they are the same for all models in the same universality class, regardless of the specific microscopic details.

In summary, while the Ising model and scalar field theory may be very different at the microscopic level, they share some fundamental properties that make them belong to the same universality class. This universality allows us to study and understand the critical behavior of these models without needing to know their microscopic details, but only their macroscopic properties.


## Intro to LQCD

In lattice QCD the concept of continuous space-time is replaced with a 4D Euclidean lattice that has a constant lattice spacing denoted as $a$. The degrees of freedom in lattice QCD are classical field variables (rather than operators) that reside on the lattice. The Euclidean action $S_E$ of the system is discretized on the lattice such that in the limit $a → 0$ the Euclidean continuum action is obtained. The Boltzmann weight factor $e ^{S_E}$ is used in subsequent computations. In order to study Euclidean correlators, the operators that appear in them are replaced by functionals that use the classical lattice field variables. The computation of Euclidean correlation functions in lattice QCD involves evaluating these functionals on a given lattice field configuration, weighting them with the Boltzmann factor, and integrating over all possible field configurations. We will mimic the whole approach in this tutorial.

### Helper functions

Let's begin by defining some useful helper functions before diving into the interesting stuff.

In [ ]:
# Look to see if you have a GPU available. If so, use it, otherwise use CPU.
torch_device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {torch_device}")
torch.set_default_device(torch_device)  # easy way but can be better if explicilty specify device below

In [ ]:
def torch_mod(x):
    return torch.remainder(x, 2*np.pi)
def torch_wrap(x):
    return torch_mod(x+np.pi) - np.pi
def grab(var):
    return var.detach().cpu().numpy()

def print_metrics(history, avg_last_N_epochs):
  print(f'== Era {era} | Epoch {epoch} metrics ==')
  for key, val in history.items():
      avgd = np.mean(val[-avg_last_N_epochs:])
      print(f'\t{key} {avgd:g}')
def moving_average(x, window=10):
    if len(x) < window:
       return np.mean(x, keepdims=True)
    else:
       return np.convolve(x, np.ones(window), 'valid') / window


We have also defined some functions for creating visually appealing plots. You may skip this part if you wish.

In [ ]:
def init_live_plot(dpi=125, figsize=(8,4)):
    fig, ax_ess = plt.subplots(1,1, dpi=dpi, figsize=figsize)
    plt.xlim(0, N_era*N_epoch)
    plt.ylim(0, 1)
    ess_line = plt.plot([0],[0], alpha=0.5) # dummy
    plt.grid(False)
    plt.ylabel('ESS')
    ax_loss = ax_ess.twinx()
    loss_line = plt.plot([0],[0], alpha=0.5, c='orange') # dummy
    plt.grid(False)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    display_id = display(fig, display_id=True)
    return dict(
        fig=fig, ax_ess=ax_ess, ax_loss=ax_loss,
        ess_line=ess_line, loss_line=loss_line,
        display_id=display_id
    )
def update_plots(history, fig, ax_ess, ax_loss, ess_line, loss_line, display_id):
    Y = np.array(history['ess'])
    Y = moving_average(Y, window=15)
    ess_line[0].set_ydata(Y)
    ess_line[0].set_xdata(np.arange(len(Y)))
    Y = history['loss']
    Y = moving_average(Y, window=15)
    loss_line[0].set_ydata(np.array(Y))
    loss_line[0].set_xdata(np.arange(len(Y)))
    ax_loss.relim()
    ax_loss.autoscale_view()
    fig.canvas.draw()
    display_id.update(fig) # need to force colab to update plot

PyTorch is a flexible machine learning framework that enables the construction of complex neural networks. However, for this particular project, we will only be using two-dimensional convolutional neural networks with varying configurations. The code below defines a fabric function that constructs a CNN with user-specified parameters.

In [ ]:
def make_conv_net(*, hidden_sizes, kernel_size, in_channels, out_channels):
    sizes = [in_channels] + hidden_sizes + [out_channels]
    #assert packaging.version.parse(torch.__version__) >= packaging.version.parse('1.5.0')
    assert kernel_size % 2 == 1, 'kernel size must be odd for PyTorch >= 1.5.0'
    padding_size = (kernel_size // 2)
    net = []
    for i in range(len(sizes) - 1):
        net.append(torch.nn.Conv2d(
            sizes[i], sizes[i+1], kernel_size, padding=padding_size,
            stride=1, padding_mode='circular'))
        if i != len(sizes) - 2:
            net.append(torch.nn.LeakyReLU())
        else:
            net.append(torch.nn.Tanh())
    return torch.nn.Sequential(*net)

As an example, we can use the following code to build a CNN with 2 hidden channels of size 8, a kernel size of 3, and 2 input channels and 4 output channels:

In [ ]:
m = make_conv_net(
    hidden_sizes=[8,8],
    kernel_size=3,
    in_channels=2,
    out_channels=4
)
print(m)

## Scalar field theory on the lattice

$\newcommand{\n}[0]{\vec{n}}$
A simple discretization of the derivatives in the continuum Euclidean action gives rise to a valid lattice Euclidean action,
\begin{equation}
\begin{split}
S^E_{\text{cont}}[\phi] &= \int d^2\vec{x} ~ (\partial_\mu \phi(\vec{x}))^2 + m^2 \phi(\vec{x})^2 + \lambda \phi(\vec{x})^4 \\
\rightarrow S^E_{\text{latt}}(\phi) &= \sum_{\n} \phi(\n) \sum_{\mu \in \{1,2\}}  \left[ 2\phi(\n) - \phi(\n+\hat{\mu}) - \phi(\n-\hat{\mu}) \right] + m^2 \phi(\n)^2 + \lambda \phi(\n)^4 \qquad (1)
\end{split}
\end{equation}
where now $\phi(\n)$ is only defined on the sites of the $L_x \times L_y$ lattice, $\vec{n} = (n_x, n_y)$, with integer $n_x, n_y$, and $\hat{\mu}$ is an operator such that $\hat{0} = (1, 0),\hat{1} = (0, 1)$. We have implicitly moved to "lattice units" where $a=1$ such that $L_x, L_y, V$ are integers and all quantities are unitless. The discretized field $\phi$ can therefore be thought of as an $(L_x \times L_y)$-dimensional vector. We use periodic boundary conditions in all directions, i.e. $\phi(L_x, y) \equiv \phi(0, y)$, etc. For convenience, we typically abbreviate $S^E_{\text{latt}} \equiv S$.

More details on $\phi^4$ lattice scalar field theory can be found in [this PhD thesis](https://edoc.hu-berlin.de/bitstream/handle/18452/14790/schroeder.pdf?sequence=1) from Ingmar Vierhaus.

### Ising limit

The action for scalar field theory defined above can be rewritten in a following form
\begin{equation}
\begin{split}
S^E_{\text{latt}}(\phi) &= \sum_{\n} -2 \kappa \sum_{\mu \in \{1,2\}} \phi(\n) \phi(\n+\hat{\mu}) + \phi(\n)^2 + \lambda' (\phi(\n)^2 - 1)^2 + constant \qquad (2)
\end{split}
\end{equation}

The Ising limit of scalar field theory is obtained by taking the limit $\lambda \rightarrow \infty$. In this limit, the scalar field is constrained to take values only in the set ${\pm 1}$, since any other value would result in an infinite potential energy contribution to the action.

To see why this is the case, consider the potential energy term in the action:

\begin{equation}
V(\phi) = \lambda'(\phi^2 - 1)^2
\end{equation}

For large $\lambda$, this potential energy term dominates over the kinetic energy term, and the field $\phi$ will tend to minimize the potential energy by taking values as close to $\pm 1$ as possible.

Substituting $\phi(\n) = \pm 1$ into the action for scalar field theory, we get:

\begin{equation}
\begin{split}
S^E_{\text{latt}}(\phi) &= \sum_{\n} -2 \kappa \sum_{\mu \in {1,2}} \phi(\n) \phi(\n+\hat{\mu}) + constant
\end{split}
\end{equation}

This is precisely the action for the two-dimensional Ising model on a lattice with coupling constant $J = 2\kappa$.

### **Exercise: rescaling bare parameters**

Although we will be using form $(1)$ of the action in our work, form $(2)$ is more commonly used in literature. In this exercise, you will need to find the relationships between the parameters $(m,\lambda)$ in eq. $(1)$ and the parameters $(\kappa,\lambda)$ in eq. $(2)$.



Answer:
$$
m^2 = \frac{1-2\lambda'}{\kappa} - 4, \, \lambda = \frac{\lambda'}{\kappa ^2}
$$

The class below implements action $(1)$. Each instance of the class is a function $S$ with fixed $m^2$, $\lambda$.

In [ ]:
class ScalarPhi4Action:
    def __init__(self, M2, lam):
        self.M2 = M2
        self.lam = lam
    def __call__(self, phi):
        action_density = torch.zeros_like(phi)
        
        # potential term
        action_density += self.M2*phi**2
        action_density += self.lam*phi**4
        
        # kinetic term (discrete Laplacian)
        Nd = len(phi.shape)-1
        dims = range(1,Nd+1)
        for mu in dims:
            action_density += 2*phi**2
            action_density -= phi*torch.roll(phi, -1, mu)
            action_density -= phi*torch.roll(phi, 1, mu)
        return torch.sum(action_density, dim=tuple(dims))

Let us return to the idea of the phase transitions in these models. Just as the Ising Model exhibits an ordered phase below the critical temperature and a disordered one above, the scalar field theory has a symmetric phase and a broken symmetry phase.
For simplicity, we restrict focus to the **symmetric phase**. The cell below instantiates an action for a set of parameters in this phase.

In [ ]:
M2 = -4.0
lam = 8.0

phi4_action = ScalarPhi4Action(M2=M2, lam=lam)

## Normalizing flows

The lattice action defines the _target distribution_ $p$ over configurations $\phi$,
\begin{equation}
p(\phi) = \frac{1}{Z} e^{-S(\phi)}, \quad
Z \equiv \int \prod_{\vec{n}} d\phi(\vec{n}) ~ e^{-S(\phi)},
\end{equation}
where $\prod_{\vec{n}}$ runs over all lattice sites $\vec{n}$.
Typically one uses algorithms like Hybrid Monte Carlo (HMC) to generate configurations from this distribution.

Instead, a powerful method to generate samples from complicated distributions is to:
1. draw samples $z$ from a simple distribution $r(z)$ (the _prior distribution_), then
2. apply a deterministic change-of-variables $f$ (a _normalizing flow_ (NF)) to get transformed samples $\phi = f(z)$.

The prior $r$ and flow $f$ together define a _normalizing flow model_. Note that in this example, $z$ and $\phi$ are both scalar fields.

The transformed samples $\phi$ are distributed according to the _model distribution_, $q$, whose density is given by the change-of-variables (or "conservation of probability") formula,

\begin{equation}
    q(\phi) = r(z) [J(z)]^{-1} = r(z) \left|\det_{kl} \frac{\partial f_k(z)}{ \partial z_l} \right|^{-1} .
\end{equation}


$f$ must be **invertible** and **differentiable** for $q(\phi)$ to be well-defined. For the model to be useful, $f$ must be expressive and the Jacobian determinant factor $J(z)$ must be efficient to calculate. As you'll see below, in some cases it is easy to compute $J$ even when the whole Jacobian matrix is intractable.

<!-- Below we construct _coupling layers_ where only the diagonal elements are needed, because the Jacobian matrix is known to be triangular. -->




### **A simple example**
The Box-Muller transform is an example of this trick in practice: to produce Gaussian random variables, draw two variables $U_1$ and $U_2$ from $\text{unif}(0,1)$ then change variables to

\begin{equation}
    Z_1 = \sqrt{-2 \ln{U_1}} \cos(2\pi U_2)
    \quad \text{and} \quad
    Z_2 = \sqrt{-2 \ln{U_1}} \sin(2\pi U_2).
\end{equation}

The resulting variables $Z_1, Z_2$ are then distributed according to an uncorrelated, unit-variance Gaussian distribution.


In [ ]:
batch_size = 2**14
u = np.random.random(size=(batch_size, 2))
z = np.sqrt(-2*np.log(u[:,0]))[:,np.newaxis] * np.stack(
    (np.cos(2*np.pi*u[:,1]), np.sin(2*np.pi*u[:,1])), axis=-1)

fig, ax = plt.subplots(1,2, dpi=125, figsize=(4,2))
for a in ax:
    a.set_xticks([-2, 0, 2])
    a.set_yticks([-2, 0, 2])
    a.set_aspect('equal')
ax[0].hist2d(u[:,0], u[:,1], bins=30, range=[[-3.0,3.0], [-3.0,3.0]])
ax[0].set_xlabel(r"$U_1$")
ax[0].set_ylabel(r"$U_2$", rotation=0, y=.46)
ax[1].hist2d(z[:,0], z[:,1], bins=30, range=[[-3.0,3.0], [-3.0,3.0]])
ax[1].set_yticklabels([])
ax[1].set_xlabel(r"$Z_1$")
ax[1].set_ylabel(r"$Z_2$", rotation=0, y=.53)
ax[1].yaxis.set_label_position("right")
ax[1].yaxis.tick_right()
plt.show()


We can analytically compute the density associated with output samples by the **change-of-variables formula** relating the _prior density_ $r(U_1, U_2) = 1$ to the _output density_ $q(Z_1, Z_2)$:

\begin{equation}
\begin{split}
    q(Z_1, Z_2) &= r(U_1, U_2) \left| \det_{kl} \frac{\partial Z_k(U_1, U_2)}{\partial U_l} \right|^{-1} \\
    &= 1 \times \left| \det \left( \begin{matrix}
        \frac{-1}{U_1 \sqrt{-2 \ln{U_1}}} \cos(2\pi U_2) &
        - 2\pi \sqrt{-2 \ln{U_1}} \sin(2\pi U_2) \\
        \frac{-1}{U_1 \sqrt{-2 \ln{U_1}}} \sin(2\pi U_2) &
        2\pi \sqrt{-2 \ln{U_1}} \cos(2\pi U_2)
        \end{matrix} \right) \right|^{-1} \\
    &= \left| \frac{2 \pi}{U_1} \right|^{-1}.
\end{split}
\end{equation}

Here, the term $J(U_1, U_2) \equiv \left| \det_{kl} \frac{\partial Z_k(U_1, U_2)}{\partial U_l} \right|$ is the determinant of the Jacobian of the transformation from $(U_1,U_2)$ to $(Z_1,Z_2)$. Intuitively, the Jacobian factor can be thought of as a change in volume element, therefore the change-of-variables formula must contain the inverse of this factor (spreading out volume decreases density). To complete the example, we can rearrange the change of variables to find $U_1 = \exp(-(Z_1^2 + Z_2^2) / 2)$ and therefore
\begin{equation}
    q(Z_1, Z_2) = \frac{1}{2\pi} e^{-(Z_1^2 + Z_2^2)/2}.
\end{equation}

**NOTE**: In this example, the model has no free parameters because we didn't need any to create a transform that exactly reproduced our target distribution (independent, unit-variance Gaussian). In general, we may not know a normalizing flow that exactly produces our desired distribution, and so instead construct parametrized models that we can variationally optimize to _approximate_ that target distribution, and because we can compute the density these can be corrected to nevertheless guarantee exactness.

### **Prior distribution**

We choose the prior distribution to be independent and identically distributed Gaussians at each lattice site, with unit width and mean zero. This is easy to sample from, and intuitively gives a "blank slate" for $f$ to build correlations into.

The cell below defines a class for this prior (which is really just a wrapper for PyTorch's normal distribution sampler), and shows how to instantiate it for lattice volume $L^2$.

You will notice that in addition to just setting up a normal and sampling, we explicitly comput the log-likelihood (aka log probability). This strategy helps with the expressiveness and tells torch that this is differentiable.

In [ ]:
class SimpleNormal:
    def __init__(self, loc, var):
        self.dist = torch.distributions.normal.Normal(
            torch.flatten(loc), torch.flatten(var))
        self.shape = loc.shape
    def log_prob(self, x):
        logp = self.dist.log_prob(x.reshape(x.shape[0], -1))
        return torch.sum(logp, dim=1)
    def sample_n(self, batch_size):
        x = self.dist.sample((batch_size,))
        return x.reshape(batch_size, *self.shape)
    
Lt, Lx = 8, 16
lattice_shape = (Lt,Lx)
prior = SimpleNormal(torch.zeros(lattice_shape), torch.ones(lattice_shape))
print("Example:",prior.sample_n(1).shape)
print("Example values: \n",prior.sample_n(1))

### Coupling layers

Expressive functions can be built through composition of simpler ones, i.e.
\begin{equation}
    f = g_n \circ g_{n-1} \circ \ldots \circ g_1 \Leftrightarrow f(z) = g_n(g_{n-1}(\ldots g_1(z)))
\end{equation}
When each simpler function is invertible and differentiable, the composed function is as well.

**Coupling layers** are one approach to defining the $g_i$ in the composed function. For input variables $\phi$, these functions are defined to update only the "active" subset $\phi_1$ conditioned on the complimentary "frozen" subset $\phi_2$.


A coupling layer $g(\phi_1, \phi_2) = (\phi_1', \phi_2')$ based on an *affine transformation* (ak transform that is just a shift, scale or stratech  see [here](https://en.wikipedia.org/wiki/Affine_transformation)), looks like

\begin{equation}
\begin{split}
  \phi_1 '  &= e^{s(\phi_2)} \phi_1 + t(\phi_2) \\
  \phi_2 '  &= \phi_2
\end{split}
\end{equation}

with inverse

\begin{equation}\begin{aligned}
  \phi_1 &= e^{-s(\phi_2')} \left( \phi_1' - t(\phi_2') \right)
  \\
  \phi_2 &= \phi_2'
\end{aligned}\end{equation}

**Note:** this equation is thinking of $\phi_1$, $s$, $t$ as vectors over the active variables/sites, and $\phi_2$ as a vector over the frozen variables/sites. The multiplication/addition is thus elementwise/per-site.

The _parameters defining the transform_, $s(\phi_2)$ and $t(\phi_2)$, can be complicated, non-invertible functions of the frozen variables $\phi_2$; **we'll use neural nets for these.**


The partioning of variables ensures that the Jacobian is triangular,

\begin{equation}
\frac{\partial g(\phi_1, \phi_2)}{\partial (\phi_1, \phi_2)} =
\left( \begin{matrix}
    \frac{\partial \phi_1'}{\partial \phi_1} & \frac{\partial \phi_1'}{\partial \phi_2} \\
    0 & 1
\end{matrix} \right).
\end{equation}

which makes for a simpler determinant that doesn't involve many combinations.

For a transform that is just a shift, scale or stratech (aka [affine transformation](https://en.wikipedia.org/wiki/Affine_transformation)), this takes the form

\begin{equation}
\frac{\partial g(\phi_1, \phi_2)}{\partial (\phi_1, \phi_2)} =
\left( \begin{array}{ccc|ccc}
    e^{[s(\phi_2)]_1} & & & \cdot & \cdot & \cdot \\
    & e^{[s(\phi_2)]_2} & & \cdot & \cdot & \cdot \\
    & & \ddots & \cdot & \cdot & \cdot \\
    \hline
    & & & 1 & & \\
    & 0 & & & 1 & \\
    & & & & & \ddots
\end{array} \right)
\end{equation}

so that the Jacobian determinant factor is just right down the diagonals

\begin{equation}
J(\phi) = \left|\det_{kl} \frac{\partial [g(\phi_1, \phi_2)]_k}{\partial (\phi_1, \phi_2)} \right| = \prod_{k} e^{[s(\phi_2)]_k}
\end{equation}

which is tractably computable.

In this example, we define the active ($\phi_1$) and frozen ($\phi_2$) subsets by assigning $\phi_{i}$ to specific elements in a 2D square array and constructing checkerboard masks that assign one square to be either active or frozen. This approach is akin to the leap-frog symplectic integrators, but now in 2D.

As a result, this allows sites to influence the transformation of their direct neighbors and build local correlations. To ensure all variables are updated, we compose coupling layers that alternatingly update odd sites and even sites, effectively a 2D leap-frog, just with a checkboards alternating active and frozen.


[**Aside:** In all the code below, for the mask $m(\vec{n}) \in \{0, 1\}$, if $m(\vec{n}) = 1$ then $\phi(\vec{n})$ is frozen and vice versa. This is just a convention.]

In [ ]:
def make_checker_mask(shape, parity):
    checker = torch.ones(shape, dtype=torch.uint8) - parity
    checker[::2, ::2] = parity
    checker[1::2, 1::2] = parity
    return checker.to(torch_device)

The code below defines an affine coupling layer. Using the formualae above for the inverse and the example of forward for the base affine transform, compute the reverse transform.

[**Aside:** class field `net` is an instance of the CNN like the ones defined above. It has one input channel (the frozen sites of the scalar field, with zeros for the active sites), and two output channels (one for $s$, one for $t$). The net produces outputs $s$ and $t$ with nonzero elements for the frozen sites, which are simply ignored using masks.]

In [ ]:
class AffineCoupling(torch.nn.Module):
    def __init__(self, net, *, mask_shape, mask_parity):
        super().__init__()
        self.mask = make_checker_mask(mask_shape, mask_parity)
        self.net = net

    def forward(self, x):
        x_frozen = self.mask * x      
        x_active = (1 - self.mask) * x
        net_out = self.net(x_frozen.unsqueeze(1))
        s, t = net_out[:,0], net_out[:,1]
        fx = (1 - self.mask) * t + x_active * torch.exp(s) + x_frozen
        axes = range(1,len(s.size()))
        logJ = torch.sum((1 - self.mask) * s, dim=tuple(axes))
        return fx, logJ

    def reverse(self, fx):
        # You will need to impelemnt this method as an exercise
        #setup masks
        #compute s,t of phi' frozen usin network
        #now compute formula above note s is s(phi') and t is t(phi')
        x =  #add code here
        logJ = #add code here
        #return x, logJ
        raise NotImplementedError()

#### **Exercise: Invertability test**

Normalizing flows are defined using a flow transformation that must be a diffeomorphism, which means that it must be both invertible and differentiable. If this condition is not met, it can result in an ill-defined posterior density and break ergodicity. Hence, using such a flow to generate samples for computations of physical observables can lead to uncontrollable bias. Although we know from mathematics that the affine flow transformation must be invertible, it is still useful to verify this numerically, as errors in our code can occur.

To ensure that flow transformation is diffeorophism we need to check that 1) flow transformation is invertable and 2) invertability of jacobian of forward and inverse transfromation. To do so we need:

1) generate random fields
$$z_0 ← \text{random()}$$
2) apply flow transformation, and compute jacobian
$$x = f(z_0), \quad \text{logJ}=\log |\det \frac{\partial z_0}{\partial x}|$$
3) apply reverse tranfromation
$$z_1 = f^{-1}(x), \quad \text{logJ1}= \text{logJ}= \log |\det \frac{\partial x}{\partial z_1}|$$
4) and verify that
$$z_1 == z_0, \quad \text{logJ} == -\text{logJ1} $$

In the previous cell, we introduced a coupling class for affine transformation. The `forward()` method of this class computes and returns the tuple `(f(z), logJ`). To ensure that the transformation is a diffeomorphism, you need to define the `reverse()` method for this class which should return `(f^{-1}(x), logJ1)`, and verify the diffeomorphism properties as outlined earlier.

In [ ]:
# EXERCISE
L = 4

# makes the coupling layer
net = make_conv_net(in_channels=1, out_channels=2, hidden_sizes=[16], kernel_size=3)
layer = AffineCoupling(net=net, mask_shape=(L,L), mask_parity=1).to(torch_device)

# makes a toy field of the appropriate size/shape
z0 = torch.arange(L**2).reshape(1,L,L).type(torch.FloatTensor).to(torch_device)

x, logJ0 = # INSERT CODE HERE
z1, logJ1 = # INSERT CODE HERE

# TODO: compare using torch.allclose
assert torch.allclose(z0, z1)
assert torch.allclose(logJ0, -logJ1)

#### **Exercise: logJ test**

Having the correct value for `logJ` is crucial in practice, as incorrect values can result in wrong gradients that prevent successful training of the model. Therefore, it is a good practice to verify the `logJ` value that is implemented in the coupling class.

In this exercise, you will need to implement a function to verify the `logJ` value and perform tests for the affine coupling class that was defined earlier. You can use the following hints to guide your implementation:

1) To simplify the computations, it is recommended to perform them for a batch size of one.

2) The `torch.autograd.grad(y_j, x)` function can be used to compute the gradients $\frac{\partial y_j}{\partial x_i}$.

3) To compute the whole Jacobian matrix `J`, you can repeat the computations for its columns `y_j`.

4) The `linalg.slogdet(J)` function can be used to obtain the required logJ value.

In [ ]:
def check_logJ(z, coupling):
    z.requires_grad = True #add this so you can use torch.autograd.grad(fx[i],z) later to get derivative
    #compute logJ and fx
    
    #Loop over fx elements and run torch.autograd.grad(fx[i],z) and add to torch J

    #Finally once you have computed J get log determinant like below
    logJ1 = np.linalg.slogdet(J)[1]
    
    #Check it works
    assert np.allclose(logJ, logJ1, atol=1e-5), \
        f'|logJ_model - logJ_torch| = {logJ - logJ1}'

check_logJ(z0, layer)

### **Composition**

The helper function below just builds a sequence of coupling layers with alternating-parity checkerboards, providing a separate neural net for each with the same hyperparameters.

In [ ]:
def make_phi4_layers(*, n_layers, lattice_shape, hidden_sizes, kernel_size):
    layers = []
    for i in range(n_layers):
        parity = i % 2
        net = make_conv_net(
            in_channels=1, out_channels=2, hidden_sizes=hidden_sizes,
            kernel_size=kernel_size)
        coupling = AffineCoupling(net, mask_shape=lattice_shape, mask_parity=parity)
        layers.append(coupling)
    return torch.nn.ModuleList(layers)

Since we will a apply many functions $f_{1}(f_{2}(x))\rightarrow f_{i}(...)$. The Jacobian factors $J_i$ from each coupling layer simply multiply together following the chain rule to define the Jacobian factor of the composed function, so that the final density is
\begin{equation}
\begin{split}
    q(x) &= r(z) \left| \det \frac{\partial f(z)}{\partial z} \right|^{-1} = r(z) \prod_{i} J_i^{-1}.
\end{split}
\end{equation}
In practice, we'll add together log Jacobians instead.

In [ ]:
def apply_flow_to_prior(prior, coupling_layers, *, batch_size):
    x = prior.sample_n(batch_size)
    logq = prior.log_prob(x)
    for layer in coupling_layers:
        x, logJ = layer.forward(x)
        logq = logq - logJ
    return x, logq

### Model

We now have everything we need to make a flow model. The cell below instantiates one.

In [ ]:
L = 16
lattice_shape = (L,L)
prior = SimpleNormal(torch.zeros(lattice_shape), torch.ones(lattice_shape))

n_layers = 16
hidden_sizes = [8,8]
kernel_size = 3
layers = make_phi4_layers(
    lattice_shape=lattice_shape, n_layers=n_layers, 
    hidden_sizes=hidden_sizes, kernel_size=kernel_size)

model = {'layers': layers, 'prior': prior}

Draw 16 configurations from the untrained flow model. What do they look like?

In [ ]:
x, logq = apply_flow_to_prior(model['prior'], model['layers'], batch_size=16)

fig,axes = plt.subplots(4,4, figsize=(5,5), sharex=True, sharey=True)
for ax, x0 in zip(axes.ravel(),x):
    ax.imshow(grab(x0).squeeze().T, aspect='equal')
    ax.grid(False)
    

Now an important question to consider is what is the appropriate function $f$ in the normalizing flow that we sample that gets us to the right answer.

Determining whether a model is a good fit for the target action is a challenging question to answer in general. However, one metric that can be used to assess model performance is the effective sample size (ESS) per configuration. In the following, we will be writing ESS but have in mind ESS per configuration.

ESS is defined on the interval $[0,1]$, and can be thought of as an effective percentage of configurations from the dataset generated from the target(fully correct) distribution that would yield the same statistics. This essentially like the efficiency of accepts when running a Markov chain sampler. For example, if a model generates 100 configurations with an ESS of 0.7, computing observables on these data would yield the same uncertainties as if you computed observables only on 70 configurations generated from the target distribution.

For MCMC, if we consider randomly sampled variables $X_{i}=\frac{p_{i}}{q_{i}}$ ie the random variable we sample to make the Markov Chain decision, we can write

$$
\rm{ESS} = \frac{N}{1+2\sum_{i}{\rm Corr}(X_{i},X_{j})}
$$
Or in other words, if we have the correct distribution $p$ the our randomly sampled $x_{i}$ have no correlation. If they are correlated, we have a biased sampler. A little bit of math, will allow us to rewrite it in a convenient way for this problem. Here, If we have the predicted distribution $q$ and the true distribution $p$ we can write this as.

$$
\frac{1}{\rm{ESS}} = \frac{\left(\sum_{i} \frac{p_{i}}{q_{i}}\right)^{2}}{N\sum_{i}\left( \frac{p_{i}}{q_{i}}\right)^{2}}\\
\frac{1}{\rm{ESS}} = \frac{1}{N}\exp\left(\log\left(\left(\sum_{i} \frac{p_{i}}{q_{i}}\right)^{2}\right) - \log\left(\sum_{i}\left( \frac{p_{i}}{q_{i}}\right)^{2}\right)\right)\\
$$

Which is what we compute below

In [ ]:
def compute_ess(logp, logq):
    logw = logp - logq
    log_ess = 2*torch.logsumexp(logw, dim=0) - torch.logsumexp(2*logw, dim=0)
    ess_per_cfg = torch.exp(log_ess) / len(logw)
    return ess_per_cfg

### **Train the model**

For convenience, the cell below instantiates the model and everything we need to train it. We'll work with very small lattices with $L=4$ so training goes quickly.

In [ ]:
# Lattice Theory
Lx, Lt = 16, 8
lattice_shape = (Lx,Lt)
M2 = -4.0
lam = 8.0

#kappa?
#lam reparametrization here?
phi4_action = ScalarPhi4Action(M2=M2, lam=lam)

# Model
prior = SimpleNormal(torch.zeros(lattice_shape, device=torch_device), torch.ones(lattice_shape, device=torch_device))

n_layers = 16
hidden_sizes = [8,8]
kernel_size = 3
layers = make_phi4_layers(
    lattice_shape=lattice_shape,
    n_layers=n_layers,
    hidden_sizes=hidden_sizes,
    kernel_size=kernel_size
    ).to(torch_device)
model = {'layers': layers, 'prior': prior}

# Training
base_lr = 0.001
optimizer = torch.optim.Adam(model['layers'].parameters(), lr=base_lr)

We need to optimize the coupling layers to match the model distribution $q(\phi)$ to the target one $p(\phi)$. To do this, we minimize a quantity known as the Kullback-Leibler (KL) divergence, which measures the distance between two distributions. Training data drawn from $p$ can be scarce in simulations of lattice field theories, so we make use of the "reverse" KL divergence,
estimated using $N$ samples drawn from the model distribution ($\phi_i \sim q$) as
\begin{equation}
\widehat{D}_{KL}(q || p) = \frac{1}{N} \sum_{i=1}^N \left[ \log{q}(\phi_i) - \log{p}(\phi_i) \right] \quad \left( \phi_i \sim q \right).
\end{equation}

We use a "reverse KL self-training" protocol that consists of
  1. Drawing samples $\phi_i$ and density estimates $\log q(\phi_i)$ from the model
  2. Computing $\log p(\phi_i) \propto -S(\phi_i)$ on each sample
  3. Using $\log q$ and $\log p$ to compute the reverse KL divergence over the samples
  3. Using standard stochastic gradient descent methods (i.e. Adam) to iteratively update neural network weights
  
  
#### **Exercise: Training Normalizing flow model**
In this exercise, you will need to implement a training function by following the algorithm outlined above. During training, we collect statistics to monitor how the training is progressing. Our built-in function will use these statistics to plot a training figure.

In [ ]:
def train_step(model, action, optimizer, metrics):
    layers, prior = model['layers'], model['prior']

    # code training
    
    # you will also need to add these metrics
    metrics['loss'].append(...)
    metrics['logp'].append(...)
    metrics['logq'].append(...)
    metrics['ess'].append(...)

Finally, let's choose hyperparameters

In [ ]:
N_era = 1
N_epoch = 200
batch_size = 1024 * 8
print_freq = 10
plot_freq = 1

history = {
    'loss' : [],
    'logp' : [],
    'logq' : [],
    'ess' : [],
    'dkl' : []
}

and run training 
(This will take a long time on a CPU but a short time on a GPU, so you want to run the next cell only for a little while on a CPU, to make sure it works, then stop it and follow the optional HPC section below it)

In [ ]:
import time
t_start = time.time()

[plt.close(plt.figure(fignum)) for fignum in plt.get_fignums()] # close all existing figures
live_plot = init_live_plot()

for era in range(N_era):
    for epoch in range(N_epoch):
        train_step(model, phi4_action, optimizer, history)

        if epoch % print_freq == 0:
            print_metrics(history, avg_last_N_epochs=print_freq)

        if epoch % plot_freq == 0:
            update_plots(history, **live_plot)

t_end = time.time()
print(f"time for execution: {(t_end - t_start):6.5f}s")

##### OPTIONAL HPC Speedup to Training

Click the arrow to the left of this section header to toggle show/hide

To get access to a GPU in an *interactive* setting, follow one of these sets of instructions.  You should be able to do this section even if you did not complete the earlier HPC section.  However, the earlier section contained more introductory text, so you may want to refer to it if you find youself with questions here.

On an HPC cluster, the main differences in the procedure compared to the earlier CPU example is, now in your `salloc` statement:

* specifies a GPU partition (machines that have a GPU)

* requests fewer CPUs (since we will only use 1)

* adds `--gres=gpu:1` (actually request 1 GPU)


Note: On many clusters, GPUs are often in shorter supply than CPUs, so please be considerate with the time you spend in a Jupyter, or other, session on a GPU ndoe.  Do whatever prep work you can do elsewhere (on CPU node or your laptop) and only start a session on the GPU node when you will actually use a GPU.  And don't forget to exit and close all terminal windows when you are done.



<details>
<summary>Instructions for running on cloud HPC Cluster (click arrow to show/hide)</summary>

##### Initial Setup

If you have not yet completed the Initial Setup once, you must do that first.  Click the arrow below for instructions.  You should only have to do this *once* this semester (unless you get a new laptop).

<details>
<summary>Initial One-Time Setup Instructions (click arrow to show/hide)</summary>

1.  Install Google Cloud CLI: [text](https://cloud.google.com/sdk/docs/install)
  - Leave the default options for the various steps (install bundled python, run `gcloud init`, etc)
  - Eventually, a browser window will pop up asking you to sign into Google giving Google Cloud SDK access to your Google Account; do this.

2.  When you are finished with the browser window that pops up, close it *AND RETURN to the terminal window* which was open before the browser, and complete initialization there.  
  - *IF* there is no longer a terminal window open, run 
  open the Google Cloud CLI link which should have been installed on your desktop and then run `gcloud init` when that opens.  Choose the following:
  - Set your default `cloud project` to: `mit-teaching-project`
  - Set your default `Compute Region and Zone` to: `us-central1-b`

3.  It may be a good idea to restart your computer now; if you do not, keep in mind to try restarting if you run into any future problems.

</details>


##### Run Jupyter on a *GPU compute node*

We will launch Jupyter notebooks in a way which will give you 2 great tools:

* manually starting a Jupyter server (on a remote machine in a custom enviornment)

* ssh port forwarding (useful even beyond Jupyter notebooks)

<details>
<summary>More Detail (click arrow to show/hide)</summary>

Some clusters may have JupyterHUB set up.  This is a tool which can automatically perform these tasks for you, but it's still good to know how to do this yourself.  JupyterHUB usually provides predefined options; with the techniques we will show you, you can request custom resources.

</details>

Following these directions, your jupyter lab session (notebooks, terminal) will have access to all of the resources (cores, memory, ...) specified in your SLURM request (the `salloc` statement).  This setup will involve you opening/using 2 different terminal windows on your laptop; we will label them `Terminal A` and `Terminal B`.

Note: if you are on a Windows machine, then by "terminal" below, we mean the window opened when you double-click on the `Google Cloud SDK Shell` icon on your Desktopm, or go to Start (Window button) and type in `Google Cloud SDK Shell`.  On a Mac, open the terminal by going to Launchpad, typing `Terminal` in Spotlight Search (CMD+Space), or in Finder open `Applications` then `Utilities` folders then click on Terminal.

In the terminal on your laptop (we will call this terminal window `Terminal A`), run the following comand (you may be prompted to enter your ssh key password)

```bash
gcloud compute ssh hpc-slurm-login-001 --tunnel-through-iap
```

<details>
<summary>If this is your first time connecting ... (click arrow to show/hide)</summary>

you may see a warning(s) like below:

```bash
WARNING: The private SSH key file for gcloud does not exist.
WARNING: The public SSH key file for gcloud does not exist.
WARNING: The PuTTY PPK SSH key file for gcloud does not exist.
WARNING: You do not have an SSH key for gcloud.
WARNING: SSH keygen will be executed to generate a key.
This tool needs to create the directory [C:\Users\muser2\.ssh] before being able to generate SSH keys.

Do you want to continue (Y/n)?
```

and/or perhaps like

```bash
The host key is not cached for this server:

compute.783492966651981141 (port 22)

You have no guarantee that the server is the computer you think it is.

The server's ssh-ed25519 key fingerprint is:

ssh-ed25519 255 SHA256:QHZjTMoUqO2yxfSn8UtkVcTpozMgvPQgfuTjmVAvQy0

If you trust this host, press "Accept" to add the key to PuTTY's cache and carry on connecting.
```

but with the numbers/characters different.  This is OK and expected.  Please continue.

</details>




<details>
<summary>If that did not work ... (click arrow to show/hide)</summary>

* If you have MIT Kerberos Ticket Manager installed, when you run the above, the connection may seem to hang (freeze).  This typically means a login window for MIT Kerberos Ticket Manager is open somewhere (often instigated by the above command).  Look to see if Kerberos Ticket Manager is open behind the terminal or other windows, or in another desktop; if so, hit cancel to close it.  Then the connection should proceede.

* If you already have Google Cloud CLI installed and use it for other projects, you will have to add these aruments to the above line to specify project name and region/zone this this course's project: `--project=mit-teaching-project --zone=us-central1-b`

</details>

This should connect you to a login node (you can confirm by typing `hostname` followed by return and it should print `hpc-slurm-login-001`).  Then run the following commands in that terminal (Note: Specific exercises instruct you to replace this `salloc` statement with a different one to request more resources)

```bash
salloc -N 1 -c 1 --partition=pgpu  --gres=gpu:1
```

This requests `1` core on 1 node.  You may have to wait for a while after the `salloc` statement for a node to become free or a new virtual node to be created.  Once you are notified your resources are ready and get a terminal prompt again, run

```bash
hostname
```

Take note of the output printed by the `hostname` command; this is the name of the compute node on which your notebook will run.  

To test that you actually have access to a GPU in this session, run the following command

```bash
nvidia-smi
```


it should output something like below.  Note, there is nothing actually running on the GPU at this point, but we see a GPU listed in the table

```bash
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      On  |   00000000:00:03.0 Off |                    0 |
| N/A   47C    P8             12W /   72W |       1MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+------------------------+----------------------+

+-----------------------------------------------------------------------------------------+
| Processes:                                                                              |
|  GPU   GI   CI        PID   Type   Process name                              GPU Memory |
|        ID   ID                                                               Usage      |
|=========================================================================================|
|  No running processes found                                                             |
+-----------------------------------------------------------------------------------------+
```


if not, please check your salloc statement to make sure you included the proper GPU-specific flags.  To try again, type `exit` Return/Enter in your cluster terminal, then try `salloc` again.


Once the above works, run the following

```bash
source /opt/venv/bin/activate
jupyter lab --no-browser --port 8080
```

The first line will activate the python virtual environment in which we have installed the necessary software.  Virtual environments are a good practice for isolating/managing packages.  The second line will start your jupyter lab server and print a lot of text to the screen, including lines like `Or copy and paste one of these URLs:` followed by a url which looks like `http://localhost:8080/lab?token=[...]` (where the `...` is a long string of letters & numbers).  Take note of this URL.

Now open a new terminal window on your laptop (we will call this `Terminal B`) and run the following command in it

```bash
gcloud compute ssh hpc-computegpu-0 --tunnel-through-iap -- -L 8080:localhost:8080
```

where instead of `hpc-computegpu-0`, write the name of the compute node (the name returned by the `hostname` command in the earlier step above).  This should just log in to the cluster and look like a regular ssh session; when it does that, just leave it open.

Now paste the URL mentioned above into your web browser on your laptop.  This should open Jupyter Lab in your web browser.  Commands you run in this Jupyter session are actually being run *on the cluster node*.

If you want to run a particular notebook on this cluster, simply drag and drop the notebook file into the Files sidebar of the Jupyter window in your browser.  (If your notebook is on colab, you will have to download your notebook, then drag the downloaded file into Jupyter Lab).

<details>
<summary>Troubleshooting (click arrow to show/hide)</summary>
  
* If you get an error that the port is not available, then please start over again substituting a different number for `8080` above.

</details>

Now return to the beginning of this section to start the exercise in this cloud cluster session.

</details>




<!-- <details>
<summary>Instructions for running on Engaging HPC Cluster (click arrow to show/hide)</summary>

For more information and troubleshooting, see: https://orcd-docs.mit.edu 
##### Initial Setup

If you have not yet completed the Initial Setup once, you must do that first.  Click the arrow below for instructions.  You should only have to do this *once* this semester (unless you get a new laptop).

<details>
<summary>Initial One-Time Setup Instructions (click arrow to show/hide)</summary>

1. Connect to the cluster by running in a terminal: `ssh <username>@orcd-login001.mit.edu` where you should replace `<username>` with your kerberos id (beginning part of your mit email address).

2. Make a private directory for this class by running the following commands

```bash
mkdir ~/private_816
chmod 700 ~/private_816
```

3. Install sofware by running

```bash

python3 -m venv ~/venv816
source ~/venv816/bin/activate
python3 -m pip install --upgrade pip
python3 -m pip install numpy scipy matplotlib tqdm imageio h5py ipykernel jupyterlab jupyter-server-proxy dask[distributed] pathos joblib ipyparallel ipywidgets torch torchvision torchaudio torchmetrics pyright python-language-server python-lsp-server

```
4.  You can log out now by running `exit` followed by Enter/Return

</details>


##### Run Jupyter on a *GPU compute node*

We will launch Jupyter notebooks in a way which will give you 2 great tools:

* manually starting a Jupyter server (on a remote machine in a custom enviornment)

* ssh port forwarding (useful even beyond Jupyter notebooks)

<details>
<summary>More Detail (click arrow to show/hide)</summary>

Some clusters may have JupyterHUB set up.  This is a tool which can automatically perform these tasks for you, but it's still good to know how to do this yourself.  JupyterHUB usually provides predefined options; with the techniques we will show you, you can request custom resources.

</details>

Following these directions, your jupyter lab session (notebooks, terminal) will have access to all of the resources (cores, memory, ...) specified in your SLURM request (the `salloc` statement).  This setup will involve you opening/using 2 different terminal windows on your laptop; we will label them `Terminal A` and `Terminal B`.

In the terminal on your laptop (we will call this terminal window `Terminal A`), run the following command but replace `<username>` with your kerberos id (beginning part of your mit email address):

```bash
ssh <username>@orcd-login001.mit.edu
```

This should connect you to a login node (you can confirm by typing `hostname` followed by return and it should print `orcd-login001`).  Then run the following command in that terminal

```bash
salloc -N 1 -p mit_quicktest -c 1
```

This requests `1` core on one node.  You may have to wait for a while after the `salloc` statement for the resources (cores on a node) to become available.  *Note:* This allocation will only last for 15 minutes!  If you want to work longer without needing to redo the steps to restablish the connection, you can replace `mit_quicktest` in the line above with `mit_normal`, but you may have to wait longer (or reduce the number of cores in the request, `-c <# cores>`).  Once a new prompt shows up, run

```bash
hostname
```

Take note of the output printed by the `hostname` command; this is the name of the compute node on which your notebook will run.  Then run the following

```bash
cd ~/private_816
source ~/venv816/bin/activate
jupyter lab --no-browser --ip=0.0.0.0  --port 8080
```

The first line will activate the python virtual environment in which you have installed the necessary software.  Virtual environments are a good practice for isolating/managing packages.  The second line will start your jupyter lab server and print a lot of text to the screen, including lines like `Or copy and paste one of these URLs:` followed by a url which looks like ` http://127.0.0.1:8080/lab?token=[...]` (where the `...` is a long string of letters & numbers).  Take note of this URL.

Now open a new terminal window on your laptop (we will call this `Terminal B`) and run the following command in it

```bash
ssh -L 8080:nodename:8080 <username>@orcd-login001.mit.edu
```

where instead of `nodename`, write the name of the compute node (the name returned by the `hostname` command in the earlier step above).

Now paste the URL mentioned above into your web browser on your laptop.  This should open Jupyter Lab in your web browser.  Commands you run in this Jupyter session are actually being run *on the cluster node*.

If you want to run a particular notebook on this cluster, simply drag and drop the notebook file into the Files sidebar of the Jupyter window in your browser.  (If your notebook is on colab, you will have to download your notebook, then drag the downloaded file into Jupyter Lab).

<details>
<summary>Troubleshooting (click arrow to show/hide)</summary>
  
* If you get an error that the port is not available, then please start over again substituting a different number for `8080` above.

</details>

Now return to the beginning of this section to start the exercise in this cloud cluster session.

</details> -->


<!-- *Note:* If running on Engaging, you will need to make the following change to **ALL** cells below where these appear

* change the line `source /opt/venv/bin/activate` to `source ~/venv816/bin/activate`
* change all references to the partition `pcpu` to `mit_quicktest` or `mit_normal`
 -->




 <details>
<summary>Instructions for running on Google Colab (click arrow to show/hide)</summary>

1. Go to https://colab.research.google.com/ and follow on-screen directions to upload this notebook.

1. From top menu `Runtime --> Change runtime type`, then select `T4 GPU` in the window that pops up, then hit `Save` button.

1.  You are now using the notebook interactively as if you were on a cluster.  Run the notebook as usual.

</details>

Once connected using the above instructions, simply run the sequence of code cells above in this section!  (Recall, you are on a new machine now, so the necessary cells preceeding the above cell must also be run).  It should complete in about 3 minutes as opposed to potentailly hours on a CPU (depending on what type of CPU you use).  This is often around a factor of `40` speedup!

*Good Practice:* To explicitly check that the GPU being used:

* On the HPC cluster in Jupyter lab: While the calculation is running, from the top menu select `File --> New --> Terminal`.  In the terminal tab that pops up, run `nvidia-smi`.  In this table, you should now see memory being used and the percentage under `GPU-Util` should be non-zero.

* On colab: If you want to "see" the GPU being used, you can select in menu `Runtime --> View resources`.  This pulls up a side panel containing GPU RAM, which should be used when you run the appropriate cells.


It is good practice to always check this way when running new code.  Often, there is a bug or missing step and the code is not actually using the GPU but running on the CPU instead.

*Batch Jobs:* To use a GPU in a batch job, simply add the new arguments in the above `salloc` statement to your `sbatch` statement, or more commonly, add the new arguments in the `salloc` statement above to new `#SBATCH ` lines in the header of your batch file.  Otherwise, follow the recipe in the above batch CPU example for porting this to batch jobs.

Congratulations!  You have now saved yourself a lot of time and have used a GPU!

##### Continuing on with **Exercise: Training Normalizing flow model**

The cell below draws a batch of configurations from the sample, computes $S(\phi)$ and $S_{\text{eff}} \equiv - \log q(\phi)$ on each one, then makes a 2d histogram comparing these quantities.

**Discuss:** What would this look like for a perfect model? How does your model compare?

In [ ]:
with torch.no_grad():
  phi, logq = apply_flow_to_prior(model['prior'], model['layers'], batch_size=1024)
S_eff = -grab(logq)
S = grab(phi4_action(phi))
fit_b = np.mean(S) - np.mean(S_eff)
print(f'slope 1 linear regression S = S_eff + {fit_b:.4f}')
fig, ax = plt.subplots(1,1, dpi=125, figsize=(4,4))
ax.hist2d(S_eff, S, bins=40, range=[[-5, 10], [-5, 10]])
ax.set_xlabel(r'$S_{\mathrm{eff}} = -\log~q(x)$')
ax.set_ylabel(r'$S(x)$')
ax.set_aspect('equal')
xs = np.linspace(-5, 10, num=4, endpoint=True)
ax.plot(xs, xs + fit_b, ':', color='w', label='slope 1 fit')
plt.legend(prop={'size': 6})
plt.show()

## Metropolis-Hastings algorithm


The Metropolis-Hastings algorithm is a Markov chain Monte Carlo (MCMC) method that generates samples for probability distributions when direct sampling is difficult or impossible. The algorithm constructs a sequence of configurations starting from an arbitrary configuration, with the sequence eventually following the target distribution. The algorithm can be summarized as follows:

1) Generate a new candidate configuration $\phi'$ from the previous configuration $\phi^{i-1}$ with a proposal probability $T(\phi^{i-1}\rightarrow \phi')$.

2) Accept the candidate configuration $\phi'$ as the new configuration $\phi^i$ with an acceptance probability:
\begin{equation}
p_{\mathrm{accept}}(\phi'|\phi^{i-1}) = \min \left(
1,
\frac{T(\phi' \rightarrow \phi^{i-1})}{T(\phi^{i-1} \rightarrow \phi')}
\frac{p(\phi')}{p(\phi^{i-1})}
\right),
\end{equation}
where $p(\phi)$ is the target probability density. If the suggested configuration is not accepted, the unchanged configuration is considered again in the sequence as $\phi^i = \phi^{i-1}$.

3) Repeat the steps to generate full ensemble.

It can be shown that samples generated with this algorithm are distributed according to the targeted probability density if $T(\phi^{i-1} \rightarrow \phi')$ is ergodic, meaning that there is a non-zero probability to generate all possible configurations $\phi'$.

### Metropolis algorithm for normalizing flow
To obtain **unbiased** estimates of observables, we can use the samples generated by our model as proposals in the Metropolis-Hastings (MH) algorithm. Using normalizing flows, we can assign a probability density $q(\phi)$ to each sample $\phi$, and this density can be used as the proposal probability in the MH algorithm. In this case, the acceptance probability takes the form:

\begin{equation}
p_{\mathrm{accept}}(\phi'|\phi^{i-1}) = \min \left(
1,
\frac{q(\phi^{i-1})}{q(\phi')}
\frac{p(\phi')}{p(\phi^{i-1})}
\right).
\end{equation}


To simplify the process, we create a function that generates and draws batches of samples from a flow model and then returns them one at a time, along with their respective $\log p$ and $\log q$ values. We can use each sample as a proposal in the MH algorithm.

In [ ]:
def serial_sample_generator(model, action, batch_size, N_samples):
    layers, prior = model['layers'], model['prior']
    layers.eval()
    x, logq, logp = None, None, None
    with torch.no_grad():
      for i in range(N_samples):
          batch_i = i % batch_size
          if batch_i == 0:
              # we're out of samples to propose, generate a new batch
              x, logq = apply_flow_to_prior(prior, layers, batch_size=batch_size)
              logp = -action(x)
          yield x[batch_i], logq[batch_i], logp[batch_i]

If you are not familiar with Python generators, this function creates an object that you can iterate over, for example.

In [ ]:
l = [phi.mean() for phi, _, _ in serial_sample_generator(model, phi4_action, 4, 6)]#vary the 6=> something what happens
print("Field values:", l)

#### **Exercise: Metroplis-Hastings for NF samples**
Now we want you to setup the full Markov Chain MC sampling using everything you trained above! You are going to have to go up and follow things carefully, but its all here. An important point to consider is the serial step should be viewed as sequential Markov Chain Proposals.

In this exercise, aka the function below, your goal is to create an MCMC ensemble using independent flow proposals and the acceptance probability formula we defined above. It's recommended to keep track of the history of some metrics, compute them, and add them to a dictionary. We will output the history later and use it to study the dynamics.

In [ ]:
def make_mcmc_ensemble(model, action, batch_size, N_samples):
    # fill this history dict with values in MCMC ensemble
    history = {
        'x' : [],  # save configurations here
        'logq' : [],
        'logp' : [],
        'accepted' : []
    }

    return history

The Independence Metropolis acceptance rate is another metric to evaluate the quality of our model, similar to the ESS.

In [ ]:
ensemble_size = 32*1024
phi4_ens = make_mcmc_ensemble(model, phi4_action, 1024, ensemble_size)
print("Accept rate:", np.mean(phi4_ens['accepted']))

The NF model generates samples with a probability density `q(x)` that only approximates the target density `p(x)` to some degree. By applying the MH algorithm, we obtain samples that are distributed exactly according to the target density. The quality of the model's approximation can be evaluated using metrics such as ESS or acceptance rate, which we previously calculated.

Next, let's visualize the bias in "raw" sample or approximation error by plotting the distribution of the action of the theory. Note that bias in different observables could be different.

In [ ]:
with torch.no_grad():
  phi, _ = apply_flow_to_prior(model['prior'], model['layers'], batch_size=ensemble_size)
phi_unbiased = torch.stack(phi4_ens['x'])

S, S_unbiased = map(lambda x: phi4_action(x).cpu(), [phi, phi_unbiased])
plt.hist(torch.stack([S, S_unbiased]), label=['model action', 'MCMC corrected action'], bins=100, density=1, histtype='step');
plt.legend();

### **Exercise (optional): training better model and examining approximation error**

The results obtained may vary depending on the quality of the model used. We encourage you to experiment with various hyperparameters, such as the CNN configuration, Adam parameters, and increasing the training duration to obtain better results. It is interesting to observe how the model samples approach the true samples as the model quality improves.

## Thermodynamics with Normalizing Flows (Not Optional...well)
### Really, this is  The Ising on the cake!

The thermodynamics of QCD has important implications for high-energy heavy-ion collision experiments, such as those performed at the Relativistic Heavy Ion Collider (RHIC) and the Large Hadron Collider (LHC). These experiments aim to recreate the conditions present in the early universe just after the Big Bang, and the study of QCD thermodynamics is essential for understanding the behavior of matter under these extreme conditions.

The thermodynamic properties of a system can be determined from the partition function
$$
Z(T,V) = \int \prod_{\vec{n}} d\phi(\vec{n}) ~ e^{-\frac{S(\phi)}{T}},
$$
where $V$ denotes the physical volume of the system and $T$ is temperature. In lattice QCD, one uses the MCMC method to generate configurations ${\phi}$ with probabilities proportional to $e^{-\frac{S(\phi)}{T}}$ and evaluate observables on these configurations. However, this method does not provide direct access to the normalizer of the probability density, $Z(T,V)$, making a study of thermodynamics a challenging problem.

Computation of patition function $Z(T,V)$ involves some approximations and tricks. Typically, these tricks rely on the fact that the ratio of partition functions  $Z(T,V)/Z(T_0,V)$ at different physical parameters $T$ and $T_0$ can be computed using samples without requiring constant normalizers. This ratio is then used to compute the partition function at the desired parameters. For instance, a possible approach would be to start from a known analytical value $Z (T_ 0, V)$. Next, one would execute MCMC simulations to compute $\frac{ Z(T,V)}{Z(T_ 0,V)}$. By combining these two results, one can obtain an interesting quantity $Z(T,V)=\frac{ Z(T,V)}{Z(T_ 0,V)} Z (T_ 0, C)$.

In practical simulations, the computation of the ratio of partition functions can be challenging due to the requirement of a good overlap of the probability densities corresponding to both partition functions in the ratio. The degree of overlap significantly decreases with changes in physical parameters, which means that the ratio can only be computed for closely related values. As a result, the interval of interest, say $[T, T_0]$, is split into a few small intervals where the overlap is good enough, and MCMC simulations are carried out for every value. This approach is computationally demanding, making the study of QCD thermodynamics a challenging problem.

Normalizing Flows provide an alternative approach. Comparing with conventional MCMC they not only generate samples $\phi$, but also provide the corresponding probability density. One can compute the partition function using the probability density provided by Normalizing Flows in a way similar to computing other observables.  This advantage allows the generation of **only one ensemble** of configurations and directly evaluates the partition function on it.  The new approach reduces the computational demand significantly and allows one to perform computations when the partition function is unknown for some base parameters.


In practical simulations, it is more common to calculate the Free Energy, which can be expressed as:
$$
F = - T \, \log Z,
$$
with $T$ being the absolute temperature. In lattice QCD with periodic boundary conditions tempreture can be expresses as $T = \frac{1}{L_t \, a}$.


### **Exercise**: Estimator of logZ

In this exercise, you will need to write an estimator for Partition function $Z$ using samples generated with probability density $q(\phi)$.

**Hint:** Start with definition of Parition function $Z$ and reweight integrand with density $q(\phi)$. After that you can use MC formula (aka average over all of $\phi$) to estimate the integral.

or more explicitly Recall :

\begin{equation}
p(\phi) = \frac{1}{Z} e^{-S(\phi)}, \quad
Z \equiv \int \prod_{\vec{n}} d\phi(\vec{n}) ~ e^{-S(\phi)},
\end{equation}

and that $q(\phi)$ approximates $p$, then by averaging over all $\phi_{i}$ we get an esimate for $Z$


### Exercise - mini-project: Free energy computation with NF

For this mini-project, your task is to train a Normalizing Flow model and compute the free energy of scalar field theory for a specific set of physical parameters (mass and coupling). You will be able to compare your results with the values obtained in arXiv:2007.07115, which will enable you to verify your work. This project closely resembles a real scientific problem, essentially, you would just need to apply all skill you learned to action of interest.

You will need:

1) Map action used in arXiv:2007.07115 to the action we used in this tutorial. Hopefully, you have already accomplished it in one of the previous exercises. We propose you to use physical parameters $\kappa=0.3$, $\lambda=0.022$ and lattice $L_t = 8$, $L_x=16$ as the smallest lattice from arXiv:2007.07115. **Hint:** when rescale ation do not forget to resacat $Z$ in a final formula.

2) Define a more expressive NF model and train it. It may require training it longer or even changing optimizer and/or optimizer hyperparameters.

3) Generate an ensemble of configurations using the MH algorithm as we did in previous sections.

4) Compute $Z$ using an estimator defined in a previous exercise. Compute free energy normalized by volume
$$
\frac{F}{V} a^2= \frac{1}{L_tL_x} \log \, Z
$$

**Hint:** Use `logsumexp` for numerical stability.

5) Estimate errors (This is Optional). Provide answers for free energy with uncertainties and compare your results with arXiv:2007.07115 Fig.2. **Hint:** Ensembles generated with MCMC often have large autocorrelations, which can result in underestimated uncertainties. Therefore, to estimate errors reliably, you can use various algorithms like  [binning](https://en.wikipedia.org/wiki/Data_binning#:~:text=Statistical%20data%20binning%20is%20a,grouping%20every%20five%20years%20together). For computing the uncertainty of a function of a random quantity, we suggest using the [bootstrap algorith](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bootstrap.html), or in otherword just taking mean and standard deviation by repeating the calculation removing some elements.  

